In [1]:
#ASE model

In [2]:
#imports:

from ase import Atom
from ase import Atoms
from ase.io import read, write
from ase.build import surface, cut
from ase.optimize import BFGS
from ase.constraints import UnitCellFilter
from ase.visualize import view
from ase.constraints import FixedLine

import pyace

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import subprocess
import re
import sys
import copy
from scipy.interpolate import griddata



In [3]:
#Safty copy of the ML_Potential: /nfshome/karanikv/TU-Darmstadt-Work/Training_Potentials/2025_03_03_ReFitting_8_embending_functions_gpunode2/potential_iteration_1559_new_kappa_0p05.yaml in nfshome/okresa/
path_to_pot = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ML_Potential"
potential_name = "potential_iteration_1559_new_kappa_0p05.yaml"

print(f'Path_to_potential: {path_to_pot}/{potential_name}')
pot = pyace.PyACECalculator(f'{path_to_pot}/{potential_name}')


Path_to_potential: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ML_Potential/potential_iteration_1559_new_kappa_0p05.yaml


In [4]:
def gb_energy(atoms, eref_Cu, eref_Ag, xAg=0):
    a, b = atoms.cell[0], atoms.cell[1]
    A = np.linalg.norm(np.cross(a,b))
    Energy_uncorrect = atoms.get_potential_energy()
    e = Corrected_Energy(Energy_uncorrect,atoms)
    return (e - len(atoms) *((1 - xAg) * eref_Cu + xAg * eref_Ag)) / (2*A)


def gb_energy_averaged(atoms, eref_Cu, eref_Ag, xAg=0):
    a, b = atoms.cell[0], atoms.cell[1]
    A = np.linalg.norm(np.cross(a,b))
    #set e_avr variable after avaraging script
    return (e_avr - len(atoms) *((1 - xAg) * eref_Cu + xAg * eref_Ag)) / (2*A)


#Energy was omitted to fit a graph and must be recorrected
def Corrected_Energy(Energy_uncorrect,atoms):
    Number_Atoms = list(atoms.symbols)
    Ag_number = Number_Atoms.count('Ag')
    Cu_number = Number_Atoms.count('Cu')
    Ga_number = Number_Atoms.count('Ga')
    Se_number = Number_Atoms.count('Se')
    
    E_Ag = -0.07060507
    E_Cu = -0.06457733
    E_Ga = -0.07974831
    E_Se = -0.16179301
    return Energy_uncorrect + Ag_number*E_Ag + Cu_number*E_Cu + Ga_number*E_Ga + Se_number*E_Se


def optimize(atoms, fmax=0.1, write=False, fname=None):
    # Constrain each atom individually along the z-direction
    indices1 = [atom.index for atom in atoms]
    #c = FixedLine(indices=indices, direction=[0, 0, 1])
    indices = list(range(len(atoms)))  # Ensure indices is a list of atom indices
    CC = []
    for index in indices:
        CC.append(FixedLine(index, [0, 0, 1]))
    atoms.set_constraint(CC)

    # Initialize and run the BFGS optimizer
    opt = BFGS(atoms)
    opt.run(fmax=fmax,steps=250)

    # Optionally write the optimized structure to a file
    if write:
        if fname:
            atoms.write("{}".format(fname), vasp5=True, direct=True, sort=True)
        else:
            atoms.write("opt.POSCAR", vasp5=True, direct=True, sort=True)

    return None


def optimize1(atoms, fmax=0.001, mask=(1, 1, 1, 0, 0, 0), write=False, fname=None):
    opt = BFGS(atoms)
    opt.run(fmax=fmax,steps=250)
    ucf = UnitCellFilter(atoms,constant_volume=True, mask=mask)
    #ucf = UnitCellFilter(atoms,constant_volume=False, mask=mask)
    opt = BFGS(ucf)
    opt.run(fmax=fmax,steps=250)
    if write:
        if fname:
            atoms.write("{}".format(fname), vasp5=True, direct=True, sort=True)
        else:
            atoms.write("opt.POSCAR", vasp5=True, direct=True, sort=True)

    return None


In [5]:
#new functions:

def get_layer_delete_input():
    # Prompt user for input
    Layer_delete_input = input("Delete a stoichiometric layer? (No[default]/left/right/custom): N/l/r/c: ").strip().lower()

    # Handle left deletion
    if Layer_delete_input == 'l':
        print("selected_side set to 'L' (Delete atom with lowest z-coordinate).")
        return "L", 1  # Set for min z-coordinate

    # Handle right deletion
    elif Layer_delete_input == 'r':
        print("selected_side set to 'R' (Delete atom with highest z-coordinate).")
        return "R", 1  # Set for max z-coordinate

    # Handle custom deletion
    elif Layer_delete_input == 'c':
        custom_input = input("Enter custom format (Nr.)_L/R (e.g., 2_L or 3_R): ").strip().upper()
        try:
            # Parse the custom input
            Nr_of_deleted_layers, selected_side = custom_input.split('_')
            print(f"selected_side: {selected_side}")
            Nr_of_deleted_layers = int(Nr_of_deleted_layers)  # Convert the number to an integer
            print(f"Nr_of_deleted_layers: {Nr_of_deleted_layers}")
            if selected_side not in ['L', 'R']:
                raise ValueError("Layer_delete should be either 'L' or 'R'.")
            print(f"selected_side set to '{selected_side.upper()}' and number of layers to delete: {Nr_of_deleted_layers}.")
            return selected_side.upper(), Nr_of_deleted_layers
        except ValueError as e:
            print(f"Invalid custom input: {e}")
            return None, None  # Return None if the custom input is invalid

    else:
        print("Skipping layer deletion.")
        return "L", 0  # No deletion


def find_and_delete_atoms(atoms, selected_side, Nr_of_deleted_layers, selection_tolerance=0.5):
    # Get the positions of all atoms
    positions = atoms.positions
    #view(atoms)

    # Run the deletion process for Nr_of_deleted_layers times
    for delete_round in range(Nr_of_deleted_layers):
        
        # Initialize variables
        target_index = None
        target_value = None

        if selected_side == "L":
            # Find the atom with the lowest z-coordinate
            target_value = float('inf')  # Start with a very large value
            for i, pos in enumerate(positions):
                z = pos[2]  # Z-coordinate is the third element in the position tuple
                if z < target_value:
                    target_value = z
                    target_index = i
            print(f"Atom with lowest z-coordinate: {atoms[target_index]} at index {target_index}")
        
        elif selected_side == "R":
            # Find the atom with the highest z-coordinate
            target_value = float('-inf')  # Start with a very low value
            for i, pos in enumerate(positions):
                z = pos[2]  # Z-coordinate is the third element in the position tuple
                if z > target_value:
                    target_value = z
                    target_index = i
            print(f"Atom with highest z-coordinate: {atoms[target_index]} at index {target_index}")
        
        else:
            print("Invalid value for selected_side. Please set it to 'L' or 'R'.")
            return None
        
        # Apply selection_tolerance: Find atoms within the tolerance range of the target z-coordinate
        atoms_to_delete = []
        for i, pos in enumerate(positions):
            z = pos[2]
            if abs(z - target_value) <= selection_tolerance:
                atoms_to_delete.append(i)

        if atoms_to_delete:
            print(f"Atoms within {selection_tolerance} tolerance of the target z-coordinate: {atoms[atoms_to_delete[0]]} to {atoms[atoms_to_delete[-1]]}")
        
        # Delete the selected atoms
        for idx in sorted(atoms_to_delete, reverse=True):  # Delete in reverse order to avoid index shifting
            print(f"Deleting atom at index {idx}: {atoms[idx]}")
            atoms.pop(idx)
        
        # Update the positions after deletion (since the index shifts after pop)
        positions = atoms.positions  # Re-fetch the updated positions

    return atoms



def find_and_swap_atoms(atoms, selected_side, Nr_of_deleted_layers, selection_tolerance=0.5, visually_destinct_element="Bi"):
    # Get the positions of all atoms
    positions = atoms.positions
    
    # Run the swap process for Nr_of_deleted_layers times
    for delete_round in range(Nr_of_deleted_layers):
        
        # Initialize variables
        target_index = None
        target_value = None

        if selected_side == "L":
            # Find the atom with the lowest z-coordinate
            target_value = float('inf')  # Start with a very large value
            for i, pos in enumerate(positions):
                z = pos[2]  # Z-coordinate is the third element in the position tuple
                if z < target_value:
                    target_value = z
                    target_index = i
            if Layer_delete_input != "skip":
                print(f"Atom with lowest z-coordinate: {atoms[target_index]} at index {target_index}")
        
        elif selected_side == "R":
            # Find the atom with the highest z-coordinate
            target_value = float('-inf')  # Start with a very low value
            for i, pos in enumerate(positions):
                z = pos[2]  # Z-coordinate is the third element in the position tuple
                if z > target_value:
                    target_value = z
                    target_index = i
            if Layer_delete_input != "skip":
                print(f"Atom with highest z-coordinate: {atoms[target_index]} at index {target_index}")
        
        else:
            print("Invalid value for selected_side. Please set it to 'L' or 'R'.")
            return None
        
        # Apply selection_tolerance: Find atoms within the tolerance range of the target z-coordinate
        atoms_to_swap = []
        for i, pos in enumerate(positions):
            z = pos[2]
            if abs(z - target_value) <= selection_tolerance:
                atoms_to_swap.append(i)

        if atoms_to_swap:
            if Layer_delete_input != "skip":
                print(f"Atoms within {selection_tolerance} tolerance of the target z-coordinate: {atoms[atoms_to_swap[0]]} to {atoms[atoms_to_swap[-1]]}")
        
        # Swap the selected atoms with new atoms of the visually distinct element
        for idx in sorted(atoms_to_swap, reverse=True):  # Iterate in reverse order to avoid index shifting
            if Layer_delete_input != "skip":
                print(f"Swapping atom at index {idx}: {atoms[idx]} with {visually_destinct_element}")
            # Replace the atom with a new one of the selected element (visually_destinct_element)
            atoms[idx].symbol = visually_destinct_element
        
        # Update the positions after swapping (though positions remain unchanged)
        positions = atoms.positions  # Re-fetch the updated positions
    
    return atoms

def delete_atoms_by_element(atoms, visually_destinct_element):
    """
    Function to delete all atoms of a specific element from the Atoms object.

    Parameters:
    atoms (ASE Atoms object): The atoms object from which atoms will be deleted.
    visually_destinct_element (str): The element symbol (e.g., 'Bi') that needs to be removed from atoms.

    Returns:
    Atoms: Updated Atoms object with the specified element deleted.
    """
    # Create a list of indices to delete
    indices_to_delete = [i for i, atom in enumerate(atoms) if atom.symbol == visually_destinct_element]
    
    # Delete the atoms by the indices
    for index in sorted(indices_to_delete, reverse=True):  # Reverse sorting to avoid index shifting
        del atoms[index]

    return atoms

In [6]:
#define new new functions (section 8 and 9)

# 1. Find the center of the atoms object
def find_center_of_atoms(atoms):
    # The center is the mean of the positions
    center = np.mean(atoms.positions, axis=0)
    print(f"Center of the atoms: {center}")
    return center

# 2. Find the [Cu] atom closest to the center
def find_closest_atom_to_center(atoms, center, atom_type):
    """
    Finds the atom of type `atom_type` closest to the center of the atoms object.

    Parameters:
    - atoms: The ASE Atoms object.
    - center: The center coordinates of the atoms object.
    - atom_type: The type of atom to search for (e.g., 'Cu').

    Prints the index and coordinates of the closest atom of the specified type to the center.
    """
    # Extract the positions and symbols of the atoms
    positions = atoms.positions
    symbols = atoms.symbols

    # Find the indices of atoms that match the specified atom_type
    atom_indices = [i for i, symbol in enumerate(symbols) if symbol == atom_type]

    # If no atoms of the specified type are found, print a message and return
    if not atom_indices:
        print(f"No atoms of type {atom_type} found.")
        return

    # Calculate distances from the center to each atom of the specified type
    distances = [np.linalg.norm(pos - center) for i, pos in enumerate(positions) if i in atom_indices]

    # Find the index of the closest atom
    closest_atom_index = atom_indices[np.argmin(distances)]
    closest_atom_position = atoms.positions[closest_atom_index]

    print(f"Closest {atom_type} atom is at index {closest_atom_index} with coordinates: {closest_atom_position}")
    return closest_atom_index, closest_atom_position


# 3. Swap the [Cu] atom closest to the center with Bi
def swap_atoms(atoms, atoms_to_swap, visually_destinct_element = "Bi"):
    """
    Swaps the selected atoms with new atoms of the visually distinct element.
    
    Parameters:
    - atoms: The list of atoms.
    - atoms_to_swap: List of atom indices to be swapped.
    - visually_destinct_element: The element to swap atoms with.
    
    Returns:
    - atoms: The modified atoms with the swapped atoms.
    """
    if isinstance(atoms_to_swap, int):
        print(f"The variable is an integer: {atoms_to_swap}")
        atoms[atoms_to_swap].symbol = visually_destinct_element
    elif isinstance(atoms_to_swap, list):
        print(f"The variable is a list: {atoms_to_swap}")
        for idx in sorted(atoms_to_swap, reverse=True):  # Iterate in reverse order to avoid index shifting
            # Replace the atom with a new one of the selected element (visually_distinct_element)
            print(f"Swapping atom at index {idx}: {atoms[idx]} with {visually_destinct_element}")
            atoms[idx].symbol = visually_destinct_element
    else:
        print("The variable is neither an integer nor a list.")

    return atoms

# Extra. Ask for a visulaisation of the GB
def ask_user_for_visuals(atoms, ask_to_check_visuals = "skip"):
    if ask_to_check_visuals != "skip":
        ask_to_check_visuals = input("Check with visualisation? y/N: ").strip().lower()
    if ask_to_check_visuals == 'y':
        #if y --> If possible manually fix it in ASE window 
        print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
        atoms.edit()
    else:
        print("Skipping visualistion.")

#4 select all atoms of one type and write them in a list: atoms_to_swap
def select_atoms_of_type(atoms, selected_atoms_type):
    """
    Selects all atoms of a specified type and returns their indices in a list.

    Parameters:
    - atoms: The ASE Atoms object.
    - selected_atoms_type: The type of atoms to select (e.g., 'Cu').
    
    Returns:
    - atoms_to_swap: A list of indices of atoms of the specified type.
    """
    # Extract the symbols of the atoms
    symbols = atoms.symbols
    
    # Find the indices of atoms that match the specified selected_atoms_type
    atoms_to_swap = [i for i, symbol in enumerate(symbols) if symbol == selected_atoms_type]

    return atoms_to_swap

    """
    # Example usage:
    atoms = Atoms('Cu3', positions=[[0, 0, 0], [1, 1, 1], [2, 2, 2]])
    # Specify the atom type you want to select
    selected_atoms_type = 'Cu'
    # Select the atoms and get their indices
    atoms_to_swap = select_atoms_of_type(atoms, selected_atoms_type)
    print(f"Indices of selected {selected_atoms_type} atoms: {atoms_to_swap}")
    """

#5 get a line of bulk_base atoms to replace:
'''
def get_atoms_with_larger_z(atoms, closest_atom_position, closest_atom_index, bulk_base = "Cu", y_tolerance = 0.1):
    """
    Finds the indices of all atoms of type `bulk_base` (e.g., 'Cu') that have a larger z-coordinate
    than the atom at `closest_atom_position` and are within `y_tolerance` in the y-coordinate.
    
    Parameters:
    - atoms: List of atom objects, each with 'element' (atom type) and 'coordinates' (x, y, z).
    - bulk_base: The atom type to search for (default is 'Cu').
    - closest_atom_position: The index of the closest atom to compare against.
    - y_tolerance: The allowed tolerance in the y-coordinate to consider the atom as within range.
    
    Returns:
    - distance_to_center_index_list: A list of indices of atoms meeting the conditions.
    """
    # define distance_to_center_index_list 
    distance_to_center_index_list = [closest_atom_index]
    # Get the coordinates of the closest atom
    #closest_atom = atoms[closest_atom_index]
    closest_atom_x = closest_atom_position[0]
    closest_atom_y = closest_atom_position[1]
    closest_atom_z = closest_atom_position[2]
    print(f"TEST 1: closest_atom_position: {closest_atom_position}")
    print(f"TEST 1: x = {closest_atom_x}, y = {closest_atom_y}, z = {closest_atom_z} ")
    # Iterate through the atoms to find matching atoms
    for idx, atom in enumerate(atoms):
        #atom(symbol='X', position=(0, 0, 0), tag=None, momentum=None, mass=None, magmom=None, charge=None, atoms=None, index=None)
        atom_x = atom.position[0]
        atom_y = atom.position[1]
        atom_z = atom.position[2]
        
         # Check if the z-coordinate of a bulk_base is larger then the center and the y-coordinate is within tolerance
        if atom.symbol == bulk_base and atom_z > closest_atom_z and abs(atom_y - closest_atom_y) <= y_tolerance:
            print(f"TEST 2: x = {atom_x}, y = {atom_y}, z = {atom_z}")
            distance_to_center_index_list_2.append(idx)      
    
    return distance_to_center_index_list_2
'''

def get_atoms_with_larger_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1, x_tolerance=0.1):
    """
    Finds the indices of all atoms of type `bulk_base` (e.g., 'Cu') that have a larger z-coordinate
    than the atom at `closest_atom_position` and are within `y_tolerance` in y and `x_tolerance` in x.

    Parameters:
    - atoms: List of atom objects, each with 'symbol' and 'position' (x, y, z).
    - bulk_base: The atom type to search for (default is 'Cu').
    - closest_atom_position: The position (x, y, z) of the reference atom.
    - closest_atom_index: The index of the reference atom (not included in result).
    - y_tolerance: The allowed tolerance in the y-coordinate.
    - x_tolerance: The allowed tolerance in the x-coordinate.

    Returns:
    - distance_to_center_index_list: A list of indices of atoms meeting the conditions.
    """
    distance_to_center_index_list = []

    closest_atom_x = closest_atom_position[0]
    closest_atom_y = closest_atom_position[1]
    closest_atom_z = closest_atom_position[2]

    print(f"Reference atom position: x={closest_atom_x}, y={closest_atom_y}, z={closest_atom_z}")

    for idx, atom in enumerate(atoms):
        if idx == closest_atom_index:
            continue  # Skip the reference atom itself

        atom_x = atom.position[0]
        atom_y = atom.position[1]
        atom_z = atom.position[2]

        if (atom.symbol == bulk_base and
            atom_z > closest_atom_z and
            abs(atom_y - closest_atom_y) <= y_tolerance and
            abs(atom_x - closest_atom_x) <= x_tolerance):
            print(f"Matching atom at index {idx}: x={atom_x}, y={atom_y}, z={atom_z}")
            distance_to_center_index_list_2.append(idx)

    return distance_to_center_index_list_2





#6 get the other line of bulk_base atoms to replace:
'''
def get_atoms_with_smaller_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1):
    """
    Finds the indices of all atoms of type `bulk_base` (e.g., 'Cu') that have a smaller z-coordinate
    than the atom at `closest_atom_position` and are within `y_tolerance` in the y-coordinate.

    Parameters:
    - atoms: List of atom objects, each with 'element' (atom type) and 'coordinates' (x, y, z).
    - bulk_base: The atom type to search for (default is 'Cu').
    - closest_atom_position: The position (x, y, z) of the closest atom to compare against.
    - closest_atom_index: The index of the closest atom.
    - y_tolerance: The allowed tolerance in the y-coordinate to consider the atom as within range.

    Returns:
    - distance_to_center_index_list: A list of indices of atoms meeting the conditions, excluding closest_atom_index.
    """
    distance_to_center_index_list = []

    # Unpack coordinates of the reference atom
    closest_atom_x = closest_atom_position[0]
    closest_atom_y = closest_atom_position[1]
    closest_atom_z = closest_atom_position[2]

    print(f"Reference atom position: x={closest_atom_x}, y={closest_atom_y}, z={closest_atom_z}")

    # Iterate through the atoms to find matching atoms
    for idx, atom in enumerate(atoms):
        if idx == closest_atom_index:
            continue  # skip the closest atom itself

        atom_x = atom.position[0]
        atom_y = atom.position[1]
        atom_z = atom.position[2]

        # Check if atom is of bulk_base type, has smaller z, and y within tolerance
        if atom.symbol == bulk_base and atom_z < closest_atom_z and abs(atom_y - closest_atom_y) <= y_tolerance:
            print(f"Matching atom at index {idx}: x={atom_x}, y={atom_y}, z={atom_z}")
            distance_to_center_index_list_1.append(idx)

    return distance_to_center_index_list_1
'''

'''
def get_atoms_with_smaller_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1):
    """
    Finds the indices of all atoms of type `bulk_base` (e.g., 'Cu') that have a smaller z-coordinate
    than the atom at `closest_atom_position` and are within `y_tolerance` in the y-coordinate.

    Parameters:
    - atoms: List of atom objects, each with 'element' (atom type) and 'coordinates' (x, y, z).
    - bulk_base: The atom type to search for (default is 'Cu').
    - closest_atom_position: The position (x, y, z) of the closest atom to compare against.
    - closest_atom_index: The index of the closest atom.
    - y_tolerance: The allowed tolerance in the y-coordinate to consider the atom as within range.

    Returns:
    - distance_to_center_index_list: A list of indices of atoms meeting the conditions.
    """
    # Initialize list with the closest atom index
    distance_to_center_index_list_2 = [closest_atom_index]
    
    # Unpack coordinates of the reference atom
    closest_atom_x = closest_atom_position[0]
    closest_atom_y = closest_atom_position[1]
    closest_atom_z = closest_atom_position[2]

    print(f"Reference atom position: x={closest_atom_x}, y={closest_atom_y}, z={closest_atom_z}")

    # Iterate through the atoms to find matching atoms
    for idx, atom in enumerate(atoms):
        atom_x = atom.position[0]
        atom_y = atom.position[1]
        atom_z = atom.position[2]

        # Check if atom is of bulk_base type, has smaller z, and y within tolerance
        if atom.symbol == bulk_base and atom_z < closest_atom_z and abs(atom_y - closest_atom_y) <= y_tolerance:
            print(f"Matching atom at index {idx}: x={atom_x}, y={atom_y}, z={atom_z}")
            distance_to_center_index_list_1.append(idx)

    return distance_to_center_index_list_1
'''

def get_atoms_with_smaller_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1, x_tolerance=0.1):
    """
    Finds the indices of all atoms of type `bulk_base` (e.g., 'Cu') that have a smaller z-coordinate
    than the atom at `closest_atom_position` and are within `y_tolerance` in y and `x_tolerance` in x.

    Parameters:
    - atoms: List of atom objects, each with 'symbol' and 'position' (x, y, z).
    - bulk_base: The atom type to search for (default is 'Cu').
    - closest_atom_position: The position (x, y, z) of the reference atom.
    - closest_atom_index: The index of the reference atom (excluded from results).
    - y_tolerance: Allowed tolerance in the y-coordinate.
    - x_tolerance: Allowed tolerance in the x-coordinate.

    Returns:
    - distance_to_center_index_list: A list of indices of atoms meeting the conditions.
    """
    distance_to_center_index_list = []

    closest_atom_x = closest_atom_position[0]
    closest_atom_y = closest_atom_position[1]
    closest_atom_z = closest_atom_position[2]

    print(f"Reference atom position: x={closest_atom_x}, y={closest_atom_y}, z={closest_atom_z}")

    for idx, atom in enumerate(atoms):
        if idx == closest_atom_index:
            continue  # Skip the center atom itself

        atom_x = atom.position[0]
        atom_y = atom.position[1]
        atom_z = atom.position[2]

        if (atom.symbol == bulk_base and
            atom_z < closest_atom_z and
            abs(atom_y - closest_atom_y) <= y_tolerance and
            abs(atom_x - closest_atom_x) <= x_tolerance):
            print(f"Matching atom at index {idx}: x={atom_x}, y={atom_y}, z={atom_z}")
            distance_to_center_index_list_1.append(idx)

    return distance_to_center_index_list_1



In [7]:
#Referece bulk energy (Eb) <--------- corrrected

ref_Cu = read("/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/CuGaSe2.POSCAR")
ref_Cu.set_calculator(pot)
optimize1(ref_Cu,write=True, fname="/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/CuGaSe2_rel.POSCAR")
eref_Cu_uncorrected = ref_Cu.get_potential_energy() / len(ref_Cu)
eref_Cu = Corrected_Energy(eref_Cu_uncorrected,ref_Cu)
print(f"\nRefecrence Bulk Energy Cu: {eref_Cu}\n")

ref_Ag = read("/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/AgGaSe2.POSCAR")
ref_Ag.set_calculator(pot)
optimize1(ref_Ag,write=True, fname="/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/AgGaSe2_rel.POSCAR")
eref_Ag_uncorrected = ref_Ag.get_potential_energy() / len(ref_Ag)
eref_Ag = Corrected_Energy(eref_Ag_uncorrected,ref_Ag)
print(f"\nRefecrence Bulk Energy Ag: {eref_Ag}\n")

print("\n\nDONE")

/tmp/ipykernel_5159/3861164029.py:4: DeprecationWarning: Please use atoms.calc = calc
  ref_Cu.set_calculator(pot)


      Step     Time          Energy          fmax
BFGS:    0 11:24:03      -60.539534        0.030474
BFGS:    1 11:24:04      -60.539633        0.026715
BFGS:    2 11:24:04      -60.539964        0.000007
      Step     Time          Energy          fmax
BFGS:    0 11:24:04      -60.539964        0.033752
BFGS:    1 11:24:04      -60.539988        0.033623
BFGS:    2 11:24:04      -60.543144        0.014454
BFGS:    3 11:24:04      -60.543166        0.012604
BFGS:    4 11:24:04      -60.543242        0.000416

Refecrence Bulk Energy Cu: -5.655599272393642

      Step     Time          Energy          fmax
BFGS:    0 11:24:04      -56.023459        0.006862
BFGS:    1 11:24:04      -56.023464        0.006095
BFGS:    2 11:24:04      -56.023483        0.000000
      Step     Time          Energy          fmax
BFGS:    0 11:24:04      -56.023483        0.019965
BFGS:    1 11:24:04      -56.023492        0.019907
BFGS:    2 11:24:04      -56.024944        0.008026
BFGS:    3 11:24:04     

/tmp/ipykernel_5159/146246128.py:58: FutureWarning: Import UnitCellFilter from ase.filters
  ucf = UnitCellFilter(atoms,constant_volume=True, mask=mask)
/tmp/ipykernel_5159/3861164029.py:11: DeprecationWarning: Please use atoms.calc = calc
  ref_Ag.set_calculator(pot)


In [8]:
#(1,-1,4)
#The distribution of grain boundary planes for the Σ9 boundary is similar: most boundaries have tilt character and the distribution maximizes at the position of the symmetric tilt.
#https://www.sciencedirect.com/science/article/pii/S1359645416305511

In [ ]:
#Functional model with strain
"Edited from Melissas script, by Alex v2.1 FUNCTIONAL again!!!"

# Define the repetition factors
repeats = [2, 4, 6, 8]

# Loop over the repetition factors
# Loop over the repetition factors
for repeat in repeats:
    #bulk = ref_Cu.copy()
    bulk = read('/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/CuGaSe2_rel.POSCAR')
    #if repeat == 2:
        #view(bulk) #[CuGaSe2 Einheitszelle]
    bulk.center()
    #if repeat == 2:
        #view(bulk)
    
    hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511        ZU VARIIREN I
    GB = surface(bulk, hkl, 36, vacuum=0.0)
    a1, b1, c1 = GB.get_cell()
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
    GB.center()

    #if repeat == 2:
    #    view(GB)

    GB = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00)) #close but not perfect
    GB.wrap()

    #if repeat == 2:
    #    view(GB)

    GB = GB.repeat((1, 1, repeat)) #[bulk size on both sides] ZU VARIIREN II und III
    GB.wrap()

    #--------

    # Flip half of it [independent of bulk size]
    pos = GB.get_scaled_positions()
    center = 0.5
    for i in range(len(pos)):
        if pos[i, 2] <= center+(0.005/(repeat/2)): #0,005 toleranzfaktor -> gefahr besteht ein Atom zu flippen/nicht zu flippen
            pos[i, 2] = -pos[i, 2] + 1*(center + (0.005/(repeat/2)) - 0.00*pos[i, 2]) #justieren zum schließen der lücke
    GB.set_scaled_positions(pos)
    #if repeat == 4:
    #    view(GB)
    #view(GB)
    C1 = GB.get_cell()
    C1[2,2] = C1[2,2] + 2.0
    GB.set_cell(C1)
    GB.center()
    GB.set_cell(GB.get_cell() - [[0, 0, 0], [0,0,0], [0,0,2]])
    GB.center()
    GB.wrap()
    #if repeat == 4:
    #   view(GB)
    #view(GB)
    
    # Save the result with a distinct filename
    path_to_LAMMPS_input = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input"
    filename = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep.vasp"
    GB.write(filename, vasp5=True, sort=True, direct=True)
    print(f"Saved: {filename}")

    #view(GB)
    
    # Convert to LAMMPS data format
    poscar_file = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep.vasp"
    atoms = read(poscar_file, format="vasp")
    mini_data_file = f"S9_mini_GB-S9-CuGaSe2-{repeat}rep.data"
    write(f"{path_to_LAMMPS_input}/mini_data_Structures/{mini_data_file}", atoms, format="lammps-data", specorder=['Ga', 'Cu', 'Ag', 'Se'])
    print(f"Saved: {mini_data_file}")
    
    #Give out Atom count and Formation Energy (maybe rewrite)
    GB.set_calculator(pot)
    Number_Atoms = list(GB.symbols)
    print(Number_Atoms.count('Ga'))
    print(Number_Atoms.count('Ag'))
    print(Number_Atoms.count('Cu'))
    print(Number_Atoms.count('Se'))
    #print(gb_energy(GB, eref_Cu, eref_Ag, xAg=0)*16.0218)

    print("\nNext")

print("\nDONE")

# Get the current date and time
import datetime # Import the 'datetime' module to work with date and time
now = datetime.datetime.now()# Create a datetime object representing the current date and time
print("\nTimestamp:")# Display a message indicating what is being printed
print(now.strftime("%d-%m-%Y %H:%M"))# Print the current date and time in a specific format "%Y-%m-%d %H:%M:%S"
# Use the 'strftime' method to format the datetime object as a string with the desired format


In [9]:
"Edited from Melissas script, by Alex v3.2 NEO SURFACE/VACCUMS version"

# Define the repetition factors
repeats = [2, 4, 6, 8]

# Loop over the repetition factors
for repeat in repeats:
    #bulk = ref_Cu.copy()
    bulk = read('/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/CuGaSe2_rel.POSCAR')
    #if repeat == 2:
        #view(bulk) #[CuGaSe2 Einheitszelle]
    bulk.center()
    #if repeat == 2:
        #view(bulk)
    
    hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511        ZU VARIIREN I
    GB = surface(bulk, hkl, 36, vacuum=0.0)
    a1, b1, c1 = GB.get_cell()
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
    GB.center()

    #if repeat == 2:
    #    view(GB)

    GB = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00)) #close but not perfect
    GB.wrap()

    #if repeat == 2:
    #    view(GB)

    GB = GB.repeat((1, 1, repeat)) #[bulk size on both sides] ZU VARIIREN II und III
    GB.wrap()

    #--------

    # Flip half of it [independent of bulk size]
    pos = GB.get_scaled_positions()
    center = 0.5
    for i in range(len(pos)):
        if pos[i, 2] <= center+(0.005/(repeat/2)): #0,005 toleranzfaktor -> gefahr besteht ein Atom zu flippen/nicht zu flippen
            pos[i, 2] = -pos[i, 2] + 1*(center + (0.005/(repeat/2)) - 0.00*pos[i, 2]) #justieren zum schließen der lücke
    GB.set_scaled_positions(pos)
    #if repeat == 4:
    #    view(GB)
    #view(GB)
    C1 = GB.get_cell()
    C1[2,2] = C1[2,2] + 2.0
    GB.set_cell(C1)
    GB.center()
    GB.set_cell(GB.get_cell() - [[0, 0, 0], [0,0,0], [0,0,2]])
    GB.center()
    GB.wrap()
    #if repeat == 4:
    #   view(GB)
    #view(GB)
    
    #--------
    
    #Add a vaccum on both sides of the cell to create a surface
    #increase cell size by 40Å in z direction
    #print(GB.get_cell())
    #GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,20]])
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,40]]) #mehr Vaccum test
    #print(GB.get_cell())
    #if repeat == 2:
    #    view(GB)
    #center the atoms to create a 20Å Vaccum on both sides of the model
    GB.center()
    #if repeat == 2:
    #    view(GB)

    #--------NEW NEW SECTION Delete 1 layer manually --> Surfaces need to be equal on both sides 
    
    if repeat == 2: #ONLY GOOD FOR 2rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[996] #"del GB[index]" 
        del GB[994] #"del GB[index]" 
        del GB[713] #"del GB[index]" 
        del GB[712] #"del GB[index]" 
        del GB[710] #"del GB[index]" 
        del GB[708] #"del GB[index]" 
        del GB[578] #"del GB[index]" 
        del GB[577] #"del GB[index]" 
        view(GB)

    if repeat == 4: #ONLY GOOD FOR 4rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[1572] #"del GB[index]" #996+576
        del GB[1570] #"del GB[index]" #994+576... idea for qick fix
        del GB[1289] #"del GB[index]" 
        del GB[1288] #"del GB[index]" 
        del GB[1286] #"del GB[index]" 
        del GB[1284] #"del GB[index]" 
        del GB[1154] #"del GB[index]" 
        del GB[1153] #"del GB[index]" 
        view(GB)

    if repeat == 6: #ONLY GOOD FOR 6rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[996+576+576] #"del GB[index]" 
        del GB[994+576+576] #"del GB[index]" 
        del GB[713+576+576] #"del GB[index]" 
        del GB[712+576+576] #"del GB[index]" 
        del GB[710+576+576] #"del GB[index]" 
        del GB[708+576+576] #"del GB[index]" 
        del GB[578+576+576] #"del GB[index]" 
        del GB[577+576+576] #"del GB[index]" 
        view(GB)

    if repeat == 8: #ONLY GOOD FOR 8rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[996+576+576+576] #"del GB[index]" 
        del GB[994+576+576+576] #"del GB[index]" 
        del GB[713+576+576+576] #"del GB[index]" 
        del GB[712+576+576+576] #"del GB[index]" 
        del GB[710+576+576+576] #"del GB[index]" 
        del GB[708+576+576+576] #"del GB[index]" 
        del GB[578+576+576+576] #"del GB[index]" 
        del GB[577+576+576+576] #"del GB[index]" 
        view(GB)

    #--------TO DO END

    #--------NEW NEW SECTION Delete 1 layer systematic approach
    '''
    #To Do: define layer
    #check if layer left = layer right
    #del smallest amount possible to arrive at layer left = layer right --> indices_to_delete
        # automize: put indices_to_delete in a list
    
      # Indices of the atoms you want to delete (you can modify this as needed)
    #indices_to_delete = [577, 578, 708, 710, 712, 713, 994, 996] #needs to be done large to small index otherwise del[2] turns 3 --> 2
    indices_to_delete = [996, 994, 713, 712, 710, 708, 578, 577] #or indices_to_delete.sort(reverse=True)
    # Delete atoms from the Atoms object GB
    GB = GB[[i for i in range(len(atoms)) if i not in indices_to_delete]] #not functional
    # Now, GB contains only the remaining atoms
    if repeat == 2:
        view(GB)  
    '''    
    #--------TO DO END

    # Save the result with a distinct filename
    path_to_LAMMPS_input = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input"
    filename = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    GB.write(filename, vasp5=True, sort=True, direct=True)
    print(f"Saved: {filename}")

    #view(GB)
    
    # Convert to LAMMPS data format
    poscar_file = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    atoms = read(poscar_file, format="vasp")
    mini_data_file = f"S9_mini_GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.data"
    write(f"{path_to_LAMMPS_input}/mini_data_Structures/{mini_data_file}", atoms, format="lammps-data", specorder=['Ga', 'Cu', 'Ag', 'Se'])
    print(f"Saved: {mini_data_file}")


    #Give out Atom count and Formation Energy (maybe rewrite)
    GB.set_calculator(pot)
    Number_Atoms = list(GB.symbols)
    print(Number_Atoms.count('Ga'))
    print(Number_Atoms.count('Ag'))
    print(Number_Atoms.count('Cu'))
    print(Number_Atoms.count('Se'))
    #print(gb_energy(GB, eref_Cu, eref_Ag, xAg=0)*16.0218)

    print("\nNext")

print("\nDONE")

# Get the current date and time
import datetime # Import the 'datetime' module to work with date and time
now = datetime.datetime.now()# Create a datetime object representing the current date and time
print("\nTimestamp:")# Display a message indicating what is being printed
print(now.strftime("%d-%m-%Y %H:%M"))# Print the current date and time in a specific format "%Y-%m-%d %H:%M:%S"
# Use the 'strftime' method to format the datetime object as a string with the desired format


Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/GB-S9-CuGaSe2-2rep_10A_VACUUMS.vasp
Saved: S9_mini_GB-S9-CuGaSe2-2rep_10A_VACUUMS.data
286
0
286
572

Next


/tmp/ipykernel_5496/1843312654.py:159: DeprecationWarning: Please use atoms.calc = calc
  GB.set_calculator(pot)


Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/GB-S9-CuGaSe2-4rep_10A_VACUUMS.vasp
Saved: S9_mini_GB-S9-CuGaSe2-4rep_10A_VACUUMS.data
574
0
574
1148

Next
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/GB-S9-CuGaSe2-6rep_10A_VACUUMS.vasp
Saved: S9_mini_GB-S9-CuGaSe2-6rep_10A_VACUUMS.data
862
0
862
1724

Next
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/GB-S9-CuGaSe2-8rep_10A_VACUUMS.vasp
Saved: S9_mini_GB-S9-CuGaSe2-8rep_10A_VACUUMS.data
1150
0
1150
2300

Next

DONE

Timestamp:
03-04-2025 15:01


In [20]:
"Edited from Melissas script, by Alex v3.2 ONLY BULK/VACCUMS version"

# Define the repetition factors
repeats = [2, 4, 6, 8]

# Loop over the repetition factors
for repeat in repeats:
    #bulk = ref_Cu.copy()
    bulk = read('/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/ASE_Scripts_input/unit_cell/CuGaSe2_rel.POSCAR')
    #if repeat == 2:
        #view(bulk) #[CuGaSe2 Einheitszelle]
    bulk.center()
    #if repeat == 2:
        #view(bulk)
    
    hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511        ZU VARIIREN I
    GB = surface(bulk, hkl, 36, vacuum=0.0)
    a1, b1, c1 = GB.get_cell()
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
    GB.center()

    #if repeat == 2:
    #    view(GB)

    GB = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00)) #close but not perfect
    GB.wrap()

    #if repeat == 2:
    #    view(GB)

    GB = GB.repeat((1, 1, repeat)) #[bulk size on both sides] ZU VARIIREN II und III
    GB.wrap()

    #Add a vaccum on both sides of the cell to create a surface
    #increase cell size by 40Å in z direction
    #print(GB.get_cell())
    #GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,20]])
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,40]]) #mehr Vaccum test
    #print(GB.get_cell())
    #if repeat == 2:
    #    view(GB)
    #center the atoms to create a 20Å Vaccum on both sides of the model
    GB.center()
    #if repeat == 2:
    #    view(GB)
    
    #--------NEW NEW SECTION Delete 1 layer manually --> Surfaces need to be equal on both sides 
    
    if repeat == 2: #ONLY GOOD FOR 2rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[420] #"del GB[index]" 
        del GB[418] #"del GB[index]" 
        del GB[137] #"del GB[index]" 
        del GB[136] #"del GB[index]" 
        del GB[134] #"del GB[index]" 
        del GB[132] #"del GB[index]" 
        del GB[2] #"del GB[index]" 
        del GB[1] #"del GB[index]" 
        #view(GB)

    if repeat == 4: #ONLY GOOD FOR 4rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[420] #"del GB[index]" 
        del GB[418] #"del GB[index]" 
        del GB[137] #"del GB[index]" 
        del GB[136] #"del GB[index]" 
        del GB[134] #"del GB[index]" 
        del GB[132] #"del GB[index]" 
        del GB[2] #"del GB[index]" 
        del GB[1] #"del GB[index]" 
        #view(GB)
 
        """
        #view(GB) !!! kein Flip --> gleicher index
        del GB[420+576] #"del GB[index]" #420+576
        del GB[418+576] #"del GB[index]" #418+576... idea for qick fix
        del GB[137+576] #"del GB[index]" 
        del GB[136+576] #"del GB[index]" 
        del GB[134+576] #"del GB[index]" 
        del GB[132+576] #"del GB[index]" 
        del GB[2+576] #"del GB[index]" 
        del GB[1+576] #"del GB[index]" 
        #view(GB)
        """
    
    
    if repeat == 6: #ONLY GOOD FOR 6rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[420] #"del GB[index]" 
        del GB[418] #"del GB[index]" 
        del GB[137] #"del GB[index]" 
        del GB[136] #"del GB[index]" 
        del GB[134] #"del GB[index]" 
        del GB[132] #"del GB[index]" 
        del GB[2] #"del GB[index]" 
        del GB[1] #"del GB[index]" 
        #view(GB)

    if repeat == 8: #ONLY GOOD FOR 8rep ver --> diffrent index for larger structures!
        #view(GB)
        del GB[420] #"del GB[index]" 
        del GB[418] #"del GB[index]" 
        del GB[137] #"del GB[index]" 
        del GB[136] #"del GB[index]" 
        del GB[134] #"del GB[index]" 
        del GB[132] #"del GB[index]" 
        del GB[2] #"del GB[index]" 
        del GB[1] #"del GB[index]" 
        #view(GB)

    #--------TO DO END

    #--------NEW NEW SECTION Delete 1 layer systematic approach
    '''
    #To Do: define layer
    #check if layer left = layer right
    #del smallest amount possible to arrive at layer left = layer right --> indices_to_delete
        # automize: put indices_to_delete in a list
    
      # Indices of the atoms you want to delete (you can modify this as needed)
    #indices_to_delete = [577, 578, 708, 710, 712, 713, 994, 996] #needs to be done large to small index otherwise del[2] turns 3 --> 2
    indices_to_delete = [996, 994, 713, 712, 710, 708, 578, 577] #or indices_to_delete.sort(reverse=True)
    # Delete atoms from the Atoms object GB
    GB = GB[[i for i in range(len(atoms)) if i not in indices_to_delete]] #not functional
    # Now, GB contains only the remaining atoms
    if repeat == 2:
        view(GB)  
    '''    
    #--------TO DO END

   
 

    # Save the result with a distinct filename
    path_to_LAMMPS_input = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input"
    filename = f"{path_to_LAMMPS_input}/POSCAR_Structures/BULK-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    GB.write(filename, vasp5=True, sort=True, direct=True)
    print(f"Saved: {filename}")

    #view(GB)
    
    # Convert to LAMMPS data format
    poscar_file = f"{path_to_LAMMPS_input}/POSCAR_Structures/BULK-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    atoms = read(poscar_file, format="vasp")
    mini_data_file = f"S9_mini_BULK-CuGaSe2-{repeat}rep_10A_VACUUMS.data"
    write(f"{path_to_LAMMPS_input}/mini_data_Structures/{mini_data_file}", atoms, format="lammps-data", specorder=['Ga', 'Cu', 'Ag', 'Se'])
    print(f"Saved: {mini_data_file}")


    #Give out Atom count and Formation Energy (maybe rewrite)
    GB.set_calculator(pot)
    Number_Atoms = list(GB.symbols)
    print(Number_Atoms.count('Ga'))
    print(Number_Atoms.count('Ag'))
    print(Number_Atoms.count('Cu'))
    print(Number_Atoms.count('Se'))
    #print(gb_energy(GB, eref_Cu, eref_Ag, xAg=0)*16.0218)

    print("\nNext")

print("\nDONE")

# Get the current date and time
import datetime # Import the 'datetime' module to work with date and time
now = datetime.datetime.now()# Create a datetime object representing the current date and time
print("\nTimestamp:")# Display a message indicating what is being printed
print(now.strftime("%d-%m-%Y %H:%M"))# Print the current date and time in a specific format "%Y-%m-%d %H:%M:%S"
# Use the 'strftime' method to format the datetime object as a string with the desired format



Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/BULK-CuGaSe2-2rep_10A_VACUUMS.vasp
Saved: S9_mini_BULK-CuGaSe2-2rep_10A_VACUUMS.data
286
0
286
572

Next


/tmp/ipykernel_5810/4173464848.py:151: DeprecationWarning: Please use atoms.calc = calc
  GB.set_calculator(pot)


Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/BULK-CuGaSe2-4rep_10A_VACUUMS.vasp
Saved: S9_mini_BULK-CuGaSe2-4rep_10A_VACUUMS.data
574
0
574
1148

Next
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/BULK-CuGaSe2-6rep_10A_VACUUMS.vasp
Saved: S9_mini_BULK-CuGaSe2-6rep_10A_VACUUMS.data
862
0
862
1724

Next
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/BULK-CuGaSe2-8rep_10A_VACUUMS.vasp
Saved: S9_mini_BULK-CuGaSe2-8rep_10A_VACUUMS.data
1150
0
1150
2300

Next

DONE

Timestamp:
24-03-2025 19:54


In [10]:
"Create a grain boundary; full custom version by Alex v4.0"

project_dir = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model"

#SECTION 1 presets:
bulk_base = "Cu"

# Define the repetition factors
# repeats = [2, 4, 6, 8]
repeats = [2]

#common visualistion section for QC
ask_to_check_visuals = "input" #"skip" --> skips the question

# Extra. Skip an entire section of the creation process:
def ask_to_skip_section(Section, skip_section = "ask"):
    #default:
    
    if skip_section == "skip":
        skip_section = "skip"
        print(f"Skipping {Section} of the model creation prosses")
    elif skip_section == "no":
        skip_section = "no"
    else:
        skip_section = input(f"Skip {Section} of the model creation prosses? y/N: ").strip().lower()
        if skip_section == "y":
            skip_section = "skip"
            print(f"Skipping {Section} of the model creation prosses")
    
    return skip_section


#SECTION 7 presets:
#Step 0: set preset values: '#OPEN !!!!!!!!!
#option: Skip all user input --> predefine #OPEN !!!!!!!!!

#presets:
#Step 1 (visualisation) preset:    #Options are: "input" "skip" 
#confirm_input1 = "input"
confirm_input1 = "skip"


#Step 2 (function) preset:         #Options are: "input" "skip" 
#Layer_delete_input = "input" 
Layer_delete_input = "skip" 
#use presets with "skip" instead of "input"
#selected side has to be L or R:
selected_side = "L"
#how many layers shoul be deleted:
Nr_of_deleted_layers = 1


#Step 3 (Safety feature) preset: WIP
define_stoichometric_layer = "skip" #Options are: "input" "skip"
stoichometric_layer_atom1 = 2 #Ga
stoichometric_layer_atom2 = 2 #Cu
stoichometric_layer_atom3 = 0 #Ag
stoichometric_layer_atom4 = 4 #Se

#step 4 (Safety feature) preset: WIP



#step 5 (function) preset:
""" already defined:
atoms 
selected_side 
Nr_of_deleted_layers 
"""
selection_tolerance = 0.5
visually_destinct_element = "Bi"


#step 6 (visualisation) preset:
#confirm_input6 = "input"
confirm_input6 = "skip"

#step 7 (function) preset:
#already done in step 5

#step 8 (visualisation) preset:
#confirm_input8 = "input"
confirm_input8 = "skip"
atom1 = "Ga"
atom2 = "Cu"
atom3 = "Ag"
atom4 = "Se"  

#step 9 (visualisation) preset:
#confirm_input9 = "input"
confirm_input9 = "skip"


# Loop over the repetition factors
for repeat in repeats:
    
    #SECTION 1 CREATE THE BULK UNIT
    
    #bulk = ref_Cu.copy()
    #bulk_base = "Cu"
    #bulk_base = "Ag"
    bulk = read(f"{project_dir}/input/ASE_Scripts_input/unit_cell/{bulk_base}GaSe2_rel.POSCAR")
    #if repeat == 2:
        #view(bulk) #[CuGaSe2 Einheitszelle]
    bulk.center()
    #if repeat == 2:
        #view(bulk)

    #SECTION 2 INCLUDE THE ORIENTATION
    
    hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511        ZU VARIIREN I
    GB = surface(bulk, hkl, 36, vacuum=0.0)
    a1, b1, c1 = GB.get_cell()
    GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
    GB.center()
    #if repeat == 2:
    #    view(GB)

    #SECTION 3 CUT THE SUPERCELL TO INCREASE READABILITY
    
    GB = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00)) #close but not perfect
    GB.wrap()

    #if repeat == 2:
    #    view(GB)

    #SECTION 4 REPEAT THE BULK UNIT
    GB = GB.repeat((1, 1, repeat)) #[bulk size on both sides] ZU VARIIREN II und III
    GB.wrap()

    #--------

    #SECTION 5 CREATE THE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE
    skip_section = ask_to_skip_section(Section = "Section 5", skip_section = "ask") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":
        # Flip half of it [independent of bulk size]
        pos = GB.get_scaled_positions()
        center = 0.5
        for i in range(len(pos)):
            if pos[i, 2] <= center+(0.005/(repeat/2)): #0,005 toleranzfaktor -> gefahr besteht ein Atom zu flippen/nicht zu flippen
                pos[i, 2] = -pos[i, 2] + 1*(center + (0.005/(repeat/2)) - 0.00*pos[i, 2]) #justieren zum schließen der lücke
        GB.set_scaled_positions(pos)
        #if repeat == 4:
        #    view(GB)
        #view(GB)
        C1 = GB.get_cell()
        C1[2,2] = C1[2,2] + 2.0
        GB.set_cell(C1)
        GB.center()
        GB.set_cell(GB.get_cell() - [[0, 0, 0], [0,0,0], [0,0,2]])
        GB.center()
        GB.wrap()
        #if repeat == 4:
        #   view(GB)
        #view(GB)
    else:
        print("This is the result of skipping section 5.")
        GB.edit()
    
    #--------
    
    #SECTION 6 CREATE A SURFACE BY INCLUDING A VACUUM
    skip_section = ask_to_skip_section(Section = "Section 6", skip_section = "no") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":
        #Add a vaccum on both sides of the cell to create a surface
        #increase cell size by 40Å in z direction
        #print(GB.get_cell())
        #GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,20]])
        GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,40]]) #mehr Vaccum test
        #print(GB.get_cell())
        #if repeat == 2:
        #    view(GB)
        #center the atoms to create a 20Å Vaccum on both sides of the model
        GB.center()
        #if repeat == 2:
        #    view(GB)
    else:
        print("This is the result of skipping section 6.")
        GB.edit()
    
    #---------------------------------------------------------------------
    
    #SECTION 7 Delete 1 layer systematic approach -> Surfaces need to be equal on both sides 
    skip_section = ask_to_skip_section(Section = "Section 7", skip_section = "no") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":
        
        #Step 1 ask: show model | visualisation
        if confirm_input1 == "skip":
            do = "nothing"
            #print("skipping Step 1. Script continues.")
        else:
            confirm_input1 = input("show model? y/N: ").strip().lower()
            
            if confirm_input1 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
            else:
                print("Step 1 Done. Script continues.")
    
    
        # Step 2 ask to delete layers: by calling function: get_layer_delete_input() | function
        if Layer_delete_input == "input": 
            selected_side, Nr_of_deleted_layers = get_layer_delete_input()
            print("Step 2 Done. Script continues.")
            #Layer_delete_input = input("Delete a stoichometric layer? (No[default]/left/right/custom): N/l/r/c")
            #custom Format: (Nr.)_L/R
        else:
            #keep preset
            do = "nothing"
            #print("Step 2 Done. Script continues.")
    
    
        #Step 3 define a stoichometric layer for comparision --> static / user input #OPEN !!!!!!!!! | Safety feature
        
        if define_stoichometric_layer == "skip":
            stoichometric_layer_list = [stoichometric_layer_atom1,stoichometric_layer_atom2,stoichometric_layer_atom3,stoichometric_layer_atom4]
        else:
            stoichometric_layer_atom1 = input(f"Atom count {atom1}:")
            stoichometric_layer_atom2 = input(f"Atom count {atom2}:")
            stoichometric_layer_atom3 = input(f"Atom count {atom3}:")
            stoichometric_layer_atom4 = input(f"Atom count {atom4}:")
            stoichometric_layer_list = [stoichometric_layer_atom1,stoichometric_layer_atom2,stoichometric_layer_atom3,stoichometric_layer_atom4]
            print(stoichometric_layer_list)
            
            print("Step 3 Done. Script continues.")
    
    
        #step 4 identify the atom index in the layer and compare to stoichometric layer #OPEN !!!!!!!!! | Safety feature
        
        #find smallest/L lagrgest/R z-Axis position
    
        #write indizes in list
        deletion_indices_list = []
        #define:
        deletion_layer_atom1 = 2 #Ga
        deletion_layer_atom2 = 2 #Cu
        deletion_layer_atom3 = 0 #Ag
        deletion_layer_atom4 = 4 #Se
        #if lists are equal: interchannge atoms with [visually_destinct_Element = Bi]
        #print("Step 4 Done. Script continues.")
    
        
        #step 5 interchannge atoms with [visually_destinct_Element = Bi] --> Large und Purple Call the function find_and_swap_atoms | function
        
        GB = find_and_swap_atoms(GB, selected_side, Nr_of_deleted_layers, selection_tolerance, visually_destinct_element)
        if Layer_delete_input != "skip":
            print("Step 5 Done. Script continues.")
    
    
        #step 6 save/show model and ask for confirmation Y/n | visualisation
    
        #ask for confirmation: # Default behavior is skip
        if confirm_input6 == "skip":
            do = "nothing"
            #print("skipping Step 6. Script continues.")
        else:
            confirm_input6 = input("Check if the correct atoms selected? y/N: ").strip().lower()
            
            if confirm_input6 == 'y':
                #step 5.5 if y --> If possible manually fix it in ASE window 
                print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
                GB.edit()
            else:
                print("Skipping correction.")
        
        #Save the result with a distinct filename #needs to be corrected
        path_to_inbetweens = f"{project_dir}/data/ASE_Inbetween_steps"
        filename = f"{path_to_inbetweens}/Custom_model_skript/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS_PLACEHOLDER_TEMP_NAME.vasp"
        GB.write(filename, vasp5=True, sort=True, direct=True)
        print(f"Saved: {filename}")
        
        if Layer_delete_input != "skip":
            print("Step 6 Done. Script continues.")
    
    
        #step 7 delete visually_destinct_Element from modell | function
        
        GB = delete_atoms_by_element(GB, visually_destinct_element)
        if Layer_delete_input != "skip":
            print("Step 7 Done. Script continues.")
    
    
        #step 8 give Total atom count and ask to show final version y/N | visualisation
    
        #Give out Atom count and Formation Energy (maybe rewrite)
        if Layer_delete_input != "skip":
            Number_Atoms = list(GB.symbols)
            Number_Atoms_count1 = Number_Atoms.count(atom1)
            Number_Atoms_count2 = Number_Atoms.count(atom2)
            Number_Atoms_count3 = Number_Atoms.count(atom3)
            Number_Atoms_count4 = Number_Atoms.count(atom4)
            print(f"Atom count {atom1}: {Number_Atoms_count1}") #Ga
            print(f"Atom count {atom2}: {Number_Atoms_count2}") #Cu
            print(f"Atom count {atom3}: {Number_Atoms_count3}") #Ag
            print(f"Atom count {atom4}: {Number_Atoms_count4}") #Se
    
        if confirm_input8 == "skip":
            do = "nothing"
            #print("skipping Step 8. Script continues.")
        else:
            confirm_input8 = input("Show deletion result? y/N: ").strip().lower()
            
            if confirm_input8 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
                print("Step 8 Done. Script continues.")
            else:
                print("Skipping correction.")
        
        
        #step 9 ask delete new Layer? y/N
    
        if confirm_input9 == "skip":
            do = "nothing"
            #print("skipping Step 9. Script continues.")
        else:
            confirm_input9 = input("delete more layers? y/N: ").strip().lower()
            
            if confirm_input9 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
            else:
                print("")
                
        if Layer_delete_input != "skip":
            if Nr_of_deleted_layers == 0:
                print(f"No layer deleted.")
            elif Nr_of_deleted_layers == 1:
                print(f"Deletion of {Nr_of_deleted_layers} layer complete.")
            else:
                print(f"Deletion of {Nr_of_deleted_layers} layers complete.")
    else:
        print("This is the result of skipping section 7.")
        GB.edit()
    
    #---------------------------------------------------------------------
    
    #SECTION 8 INCLUDE AG OR VACANCYS

    # Step 1 index Cu center to border:
    # Problem: Shallow Copy: GB_revert = GB creates a shallow copy of GB. 
    # This means both GB_revert and GB reference the same object (atoms list).
    # Solution:
    # Create a deep copy of GB
    GB_revert = copy.deepcopy(GB)  # Make sure GB_revert is an independent copy

    # Step 1.1 find center
    center = find_center_of_atoms(GB)
    
    # Step 1.2 find Cu pos closest to center
    closest_atom_index, closest_atom_position = find_closest_atom_to_center(GB, center, "Cu")
    print(f"TEST: Closest Cu atom is at index {closest_atom_index} with coordinates: {closest_atom_position}")
    
    # Step 1.2.1 confirm Cu (replace with Bi)
    GB_test1 = swap_atoms(GB, closest_atom_index, "Bi")
    print(f"ask_user_for_visuals for Step: 1.2.1")
    ask_user_for_visuals(GB_test1, ask_to_check_visuals)
    
    # Reset the symbols to the original (using the deep copy)
    GB = copy.deepcopy(GB_revert)  # Revert back to the original state

    #Step 1.3 find Cu pos > center (in x or y tolerance room? 1 [x] strict \ \ \ \ pattern should not matter --> periodicity)
    #Step 1.4 create index list: distance_to_center_index_list 

    #def get_atoms_with_smaller_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1):
    distance_to_center_index_list_1 = get_atoms_with_smaller_z(GB, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance= 0.16)
    print(f"distance_to_center_index_list_part1 = {distance_to_center_index_list_1}")

    distance_to_center_index_list_2 = get_atoms_with_larger_z(GB, closest_atom_position, closest_atom_index, bulk_base = "Cu", y_tolerance = 0.16)    
    print(f"distance_to_center_index_list_part2 = {distance_to_center_index_list_2}")

    distance_to_center_index_list = distance_to_center_index_list_1 + distance_to_center_index_list_2
    print(f"distance_to_center_index_list = {distance_to_center_index_list}")
    
    #Step 1.5 test version --> untill surface is hit (save atoms object as GB_incl_long)
    GB_test2 = swap_atoms(GB, distance_to_center_index_list, "Bi")
    view(GB_test2)
    print(f"ask_user_for_visuals for Step: 1.5")
    ask_user_for_visuals(GB_test2, ask_to_check_visuals)
    # Reset the symbols to the original (using the deep copy)
    GB = copy.deepcopy(GB_revert)  # Revert back to the original state
    #print("edit 3")
    #GB.edit()

    #Step 1.6.1 cut index list down to half of the remaining cell --> at once
    #distance_to_center_index_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    # Calculate the halfway point
    halfway_index = len(distance_to_center_index_list) // 2
    # Slice the list to get the first half
    first_half = distance_to_center_index_list[:halfway_index]
    print("First half of the list:", first_half)
    GB_test3 = swap_atoms(GB, first_half, "Bi")
    view(GB_test3)
    print(f"ask_user_for_visuals for Step: 1.6.1")
    ask_user_for_visuals(GB_test3, ask_to_check_visuals)
    # Reset the symbols to the original (using the deep copy)
    GB = copy.deepcopy(GB_revert)  # Revert back to the original state
    #print("edit 4")
    #GB.edit()

    #Step 1.6.2 cut index list down to half of the remaining cell --> periodicity every 2nd
    print(distance_to_center_index_list)
    half_list = [x for x in distance_to_center_index_list if x % 2 != 0]   
    print(half_list)
    GB_test4 = swap_atoms(GB, half_list, "Bi")
    #view(GB_test4)
    print(f"ask_user_for_visuals for Step: 1.6.2")
    ask_user_for_visuals(GB_test3, ask_to_check_visuals)
    # Reset the symbols to the original (using the deep copy)
    GB = copy.deepcopy(GB_revert)  # Revert back to the original state
    #print("edit 5")
    #GB.edit()
    
    print("\n\nExecution is being cancelled 2. Breaking the script.")
    sys.exit()  # Exit the script immediately
    
    inclusion_atom_type = "Ag"

    #Step 3
    #systematicly SWAP replace 1 + save vasp [Ag]
    GB = swap_atoms(GB, distance_to_center_index_list, inclusion_atom_type)
    print(f"ask_user_for_visuals for Step: 3.4")
    ask_user_for_visuals(GB, ask_to_check_visuals)

    #Step 3.5
    #systematicly DEL delete 1 + save vasp [vac]
    GB = delete_atoms_by_element(GB, inclusion_atom_type)
    print(f"ask_user_for_visuals for Step: 3.5") 
    ask_user_for_visuals(GB, ask_to_check_visuals)

    #include: GB.delete_atoms_by_element(GB , incl = "Ag")

    
    #---------------------------------------------------------------------

    #SECTION 9 INCLUDE xAG OR xCu

    #Step 1 Select all Cu --> index in list
    replace_index_list = select_atoms_of_type(atoms, "Cu")

    #Step 2 Select x% with randomizer (seeded)


    #Step 3 SWAP
    GB = swap_atoms(GB, replace_index_list, inclusion_atom_type)
    ask_user_for_visuals(GB, ask_to_check_visuals)

    #Step 4 save as
    


    #-------------------------------------------------------------------------

    # Save the result with a distinct filename
    path_to_LAMMPS_input = f"{project_dir}/input/LAMMPS_input/Active_input"
    filename = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    GB.write(filename, vasp5=True, sort=True, direct=True)
    print(f"Saved: {filename}")

    #view(GB)
    
    # Convert to LAMMPS data format
    poscar_file = f"{path_to_LAMMPS_input}/POSCAR_Structures/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.vasp"
    atoms = read(poscar_file, format="vasp")
    mini_data_file = f"S9_mini_GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.data"
    write(f"{path_to_LAMMPS_input}/mini_data_Structures/{mini_data_file}", atoms, format="lammps-data", specorder=['Ga', 'Cu', 'Ag', 'Se'])
    print(f"Saved: {mini_data_file}")

    #Give out Atom count and Formation Energy (maybe rewrite)
    Number_Atoms = list(GB.symbols)
    Number_Atoms_count1 = Number_Atoms.count(atom1)
    Number_Atoms_count2 = Number_Atoms.count(atom2)
    Number_Atoms_count3 = Number_Atoms.count(atom3)
    Number_Atoms_count4 = Number_Atoms.count(atom4)
    print(f"Atom count {atom1}: {Number_Atoms_count1}") #Ga
    print(f"Atom count {atom2}: {Number_Atoms_count2}") #Cu
    print(f"Atom count {atom3}: {Number_Atoms_count3}") #Ag
    print(f"Atom count {atom4}: {Number_Atoms_count4}") #Se
    #GB.set_calculator(pot)
    #print(gb_energy(GB, eref_Cu, eref_Ag, xAg=0)*16.0218)

    print(f"{repeat}rep done\n\nNext:")

print("\nDONE")

# Get the current date and time
import datetime # Import the 'datetime' module to work with date and time
now = datetime.datetime.now()# Create a datetime object representing the current date and time
print("\nTimestamp:")# Display a message indicating what is being printed
print(now.strftime("%d-%m-%Y %H:%M"))# Print the current date and time in a specific format "%Y-%m-%d %H:%M:%S"
# Use the 'strftime' method to format the datetime object as a string with the desired format


Skip Section 5 of the model creation prosses? y/N:  


Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/GB-S9-CuGaSe2-2rep_10A_VACUUMS_PLACEHOLDER_TEMP_NAME.vasp
Center of the atoms: [  4.01002509   9.80251975 103.70002872]
Closest Cu atom is at index 132 with coordinates: [  5.00378349  10.95411366 103.60000186]
TEST: Closest Cu atom is at index 132 with coordinates: [  5.00378349  10.95411366 103.60000186]
The variable is an integer: 132
ask_user_for_visuals for Step: 1.2.1


Check with visualisation? y/N:  


Skipping visualistion.
TEST 1: closest_atom_position: [  5.00378349  10.95411366 103.60000186]
TEST 1: x = 5.003783490455087, y = 10.954113661091915, z = 103.60000185801674 
TEST 2: x = 7.005296886637099, y = 10.960396202701597, z = 107.2726240456237
TEST 2: x = 5.003783490455058, y = 10.966678744311276, z = 110.7437836830641
TEST 2: x = 7.005296886637106, y = 10.972961285920947, z = 114.21494332050447
TEST 2: x = 5.003783490455094, y = 10.979243827530583, z = 117.68610295794485
TEST 2: x = 7.005296886637096, y = 10.985526369140269, z = 121.15726259538516
TEST 2: x = 5.003783490455065, y = 10.991808910749949, z = 124.62842223282557
TEST 2: x = 7.005296886637106, y = 10.998091452359622, z = 128.09958187026595
TEST 2: x = 5.003783490455065, y = 11.00437399396926, z = 131.57074150770632
TEST 2: x = 7.005296886637089, y = 11.010656535578947, z = 135.04190114514665
TEST 2: x = 5.003783490455062, y = 11.01693907718862, z = 138.51306078258705
TEST 2: x = 7.005296886637089, y = 11.023221618798

Check with visualisation? y/N:  


...

Check with visualisation? y/N:  y


Script will continue after closing the ASE gui window.
Edits will be saved
please correct the deletion layer using the Ctrl+Y feature in ASE
[132, 711, 725, 734, 748, 757, 772, 781, 796, 805, 820, 829, 844, 853, 868, 877, 892, 901, 916, 925, 940, 949, 964, 973]
[711, 725, 757, 781, 805, 829, 853, 877, 901, 925, 949, 973]
The variable is a list: [711, 725, 757, 781, 805, 829, 853, 877, 901, 925, 949, 973]
Swapping atom at index 973: Atom('Cu', [7.005296886637138, 11.098612118114294, 183.63813606931186], index=973) with Bi
Swapping atom at index 949: Atom('Cu', [7.005296886637053, 11.08604703489495, 176.69581679443107], index=949) with Bi
Swapping atom at index 925: Atom('Cu', [7.005296886637096, 11.07348195167562, 169.7534975195504], index=925) with Bi
Swapping atom at index 901: Atom('Cu', [7.005296886637103, 11.060916868456282, 162.81117824466958], index=901) with Bi
Swapping atom at index 877: Atom('Cu', [7.005296886637089, 11.04835178523695, 155.8688589697889], index=877) with Bi
Sw

Check with visualisation? y/N:  


Skipping visualistion.


Execution is being cancelled 2. Breaking the script.


SystemExit: 

In [19]:
"Create a grain boundary; full custom version by Alex v14_04-20:00" 
"FINAL VERSION"

#used function


#loop feature: DONE 2done 3done 5done 6done 9done


project_dir = "/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model"

print_defaults = True

#presets: all

#option: Skip all user input --> predefine #OPEN !!!!!!!!!
#skip sections presets:
#options are: "ask" "skip" "no" "edit"
#[1   ,2   ,3   ,4   ,5   ,6   ,7   ,8   ,9   ]
#[null,"no","no","ask","no","no","no","no","no"]

#save presets in log
#read out logs
#save presets in table

#OPEN: SECTION 9

#input feature: OPEN 1 2done(1/2) 3done 4 5 6 7 8 9

#--------------------------------------------------------------------------------------------------
#saving system: DONE 3 4 5 6 7 8 9

#Reset defaults:
Section = None
print_save_path = True
custom_filename = False
extend_filename = None
save_in = "input"
# filename creation params (pass in from global or section-specific logic)
bulk_base = None     
flipped = None
gb_type = None
tilt_angle = None
new_pattern = True
repeats = None
repeat = None
layer_nr = None
wideness = None
wide = None
vacuums = None
position_index = None
inclusion_atom_type = None
xincl = None

#def create_filename()

#naming sceme: {bulk_base:Cu/Ga}GaSe2_{Flipped:GB/BULK}_{tilt_angle:Degree/S3/S9/etc.}_{repete}rep_{wide:{}on_default/<value>wide-else}
#              _{}on_default-or-XXVACUUMS_{}on_default-or-PosX_{incl:Ag/Vac/Cu}-->index:atom_delta_center_{}on_default-or-XX{incl:Ag/Cu}.data

#Safe file name part1: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_ 
#or
#Safe file name part1: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{layer_nr:repeat*layers}-z-layers
#Safe file name part2:
#section2extra: {}on_default-or-{tilt_angle_2}
#section4extra: {}on_default-or-{wide}{_}--and-- !!! Rep count: xy_atoms_wide?
#section5extra: {}on_default-or-XXVACUUMS_--and--
#section6extra: {}on_default-or-PosX-{inclusion_atom_type}-->index:atom_delta_center_
#section9extra: {}on_default-or-XX{inclusion_atom_type}.data
# how empty on None instead of string

def create_filename(
    bulk_base="Cu",                      # "Cu" or "Ga"
    flipped="GB",                        # "GB" or "BULK"
    gb_type="S9",                        # e.g., "S3", "S9", etc.
    tilt_angle="filler",                 # optional; fallback if gb_type is not set
    new_pattern=True,                    # switch between z-layers or repeat
    repeat=2,                            # repeat count
    layers=36,                           # layer count
    layer_nr=None,                       # repeat*layer count used if new_pattern is True
    wide=None,                           # e.g., "5", or None for default
    vacuums=None,                        # e.g., "20", or None for default
    position_index=None,                 # e.g., "4", or None for default
    inclusion_atom_type=None,            # e.g., "Ag", "Cu", "Vac", or None
    xincl=None,                          # e.g., "50", or None for default
    extend_filename=None                 # additional tag, optional
):
    
    parts = []

    # Part 1
    parts.append(f"{bulk_base}GaSe2_{flipped}")

    # Part 2
    if gb_type:
        parts.append(gb_type)
    elif tilt_angle:
        parts.append(tilt_angle)

    

    # Part 3
    if new_pattern:
        if layer_nr == None:
            parts.append(f"{layers}z-layers")
        else:
            parts.append(f"{layer_nr}z-layers")
    else:
        parts.append(f"{repeat}rep")

    # Part 4 - Extras
    if wide:
        parts.append(f"{wide}wide")
    if vacuums:
        parts.append(f"{vacuums}A-VACUUMS")
    if position_index and inclusion_atom_type:
        parts.append(f"Pos{position_index}-{inclusion_atom_type}")
    if xincl and inclusion_atom_type:
        parts.append(f"{xincl}{inclusion_atom_type}")
    if extend_filename:
        parts.append(extend_filename)

    # Extension
    filename = "_".join(parts) + ".vasp"

    return filename

#save_in options: 
#"input","default","both","none","active_only" ["d","b","n","a"]

#def save_functionality(Section ,print_save_path = True ,custom_filename = False, extend_filename = "" , save_in = "input"):
def save_functionality(
    Section,
    print_save_path=True,
    custom_filename=False,
    extend_filename="",
    save_in="input",
    # filename creation params (pass in from global or section-specific logic)
    bulk_base="Cu",                      #1 "Cu" or "Ga"
    flipped="GB",                        #5 "GB" or "BULK"
    gb_type="S9",                        #2 e.g., "S3", "S9", etc.
    tilt_angle="filler",                 #2 optional; fallback if gb_type is not set
    new_pattern=True,                    #4 switch between z-layers or repeat
    repeat=2,                            #4 repeat count
    layers=36,                           #2 layer count
    layer_nr=36,                         #4 repeat*layer count used if new_pattern is True
    wide=None,                           #4 e.g., "5", or None for default
    vacuums=None,                        #6 e.g., "20", or None for default
    position_index=None,                 #8 e.g., "4", or None for default
    inclusion_atom_type=None,            #8 e.g., "Ag", "Cu", "Vac", or None
    xincl=None,                          #9 e.g., "50", or None for default
):

    if save_in == "input":
        save_in = input(f'Save {Section} of the model creation prosses in:\n"default","both","none","active_only" D/b/n/a:').strip().lower()
        if save_in == "n":
            save_in = "skip"
            print(f"Skipping saving of {Section} of the model creation prosses")
        elif save_in == "a":
            save_in = "active"
        elif save_in == "b":
            save_in = "both"
        else:
            save_in = "default"

    #Save into deafult:
    if save_in == "default" or save_in == "both": 
        path_to_inbetweens = f"{project_dir}/data/ASE_Inbetween_steps"
        #filename = f"GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS_PLACEHOLDER_TEMP_NAME{extend_filename}.vasp"
        filename = create_filename(bulk_base=bulk_base, flipped=flipped, gb_type=gb_type, tilt_angle=tilt_angle,
                                   new_pattern=new_pattern, repeat=repeat, layers=layers, layer_nr=layer_nr, wide=wide, vacuums=vacuums,
                                   position_index=position_index, inclusion_atom_type=inclusion_atom_type, xincl=xincl,
                                   extend_filename=extend_filename)
        if custom_filename == True:
            filename = input(f'Input custom filename:').strip()
            print(f"filename is: {custom_filename}")
        filepath = f"{path_to_inbetweens}/Custom_model_skript/{Section}/{filename}"
        # Make sure the folder exists before saving
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        GB.write(filepath, vasp5=True, sort=True, direct=True)
        
        if print_defaults == True and print_save_path == True:
            print(f"Saved: {filepath}")
    
    elif save_in == "active" or save_in == "both":
        # Save the result with a distinct filename
        path_to_LAMMPS_input = f"{project_dir}/input/LAMMPS_input/Active_input"
        #filename = f"GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS{extend_filename}.vasp"
        filename = create_filename(bulk_base=bulk_base, flipped=flipped, gb_type=gb_type, tilt_angle=tilt_angle,
                                   new_pattern=new_pattern, repeat=repeat, layers=layers, layer_nr=layer_nr, wide=wide, vacuums=vacuums,
                                   position_index=position_index, inclusion_atom_type=inclusion_atom_type, xincl=xincl,
                                   extend_filename=extend_filename)
        if custom_filename == True:
            filename = input(f'Input custom filename:').strip()
            print(f"filename is: {custom_filename}")
        filepath = f"{path_to_LAMMPS_input}/POSCAR_Structures/{filename}"
        # Make sure the folder exists before saving
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        GB.write(filepath, vasp5=True, sort=True, direct=True)
        if print_save_path == True:
            print(f"Saved: {filepath}")
    
        # Convert to LAMMPS data format
        poscar_file = filepath
        atoms = read(poscar_file, format="vasp")
        #mini_data_filename = f"S9_mini_GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS.data"
        mini_data_filename = create_filename(bulk_base=bulk_base, flipped=flipped, gb_type=gb_type, tilt_angle=tilt_angle,
                                             new_pattern=new_pattern, repeat=repeat, layer_nr=layer_nr, wide=wide, vacuums=vacuums,
                                             position_index=position_index, inclusion_atom_type=inclusion_atom_type, xincl=xincl,
                                             extend_filename="mini")
        write(f"{path_to_LAMMPS_input}/mini_data_Structures/{mini_data_filename}", atoms, format="lammps-data", specorder=['Ga', 'Cu', 'Ag', 'Se'])
        if print_save_path == True:
            print(f"Saved: {mini_data_filename}")

    else:
        do = "nothing"
        print(f"Skipping saving of {Section} of the model creation prosses")

    #return save_in

#Wrapper for save_functionality()
#def save_model(Section, print_save_path=True, custom_filename=False, extend_filename="", save_in="input"):
def save_model(
    Section,
    print_save_path=True,
    custom_filename=False,
    extend_filename="",
    save_in="input"
):
    # Grab values from global scope, or set defaults if not defined
    bulk_base = globals().get("bulk_base", "Cu")
    flipped = globals().get("flipped", "GB")
    gb_type = globals().get("gb_type", "S9")
    tilt_angle = globals().get("tilt_angle", None)
    new_pattern = globals().get("new_pattern", True)
    repeat = globals().get("repeat", 2)
    layers = globals().get("layers", 999)
    layer_nr = globals().get("layer_nr", None)
    wide = globals().get("wide", None)
    vacuums = globals().get("vacuums", None)
    position_index = globals().get("position_index", None)
    inclusion_atom_type = globals().get("inclusion_atom_type", None)
    xincl = globals().get("xincl", None)

    # Call main function with all these values
    save_functionality(
        Section=Section,
        print_save_path=print_save_path,
        custom_filename=custom_filename,
        extend_filename=extend_filename,
        save_in=save_in,
        # filename creation params (pass in from global or section-specific logic)
        bulk_base=bulk_base,
        flipped=flipped,
        gb_type=gb_type,
        tilt_angle=tilt_angle,
        new_pattern=new_pattern,
        repeat=repeat,
        layers=layers,
        layer_nr=layer_nr,
        wide=wide,
        vacuums=vacuums,
        position_index=position_index,
        inclusion_atom_type=inclusion_atom_type,
        xincl=xincl
    )
#--------------------------------------------------------------------------------------------------

# Extra. Skip an entire section of the creation process:
def ask_to_skip_section(Section, skip_section = "ask"):
    #default:
    
    if skip_section == "skip":
        skip_section = "skip"
        print(f"Skipping {Section} of the model creation prosses")
    elif skip_section == "no":
        skip_section = "no"
    elif skip_section == "edit":
        skip_section = "edit"
    else:
        skip_section = input(f"Skip {Section} of the model creation prosses? y/N: ").strip().lower()
        if skip_section == "y":
            skip_section = "skip"
            print(f"Skipping {Section} of the model creation prosses")
        if skip_section == "e":
            skip_section = "edit"
    
    return skip_section

def Confirmation_question(custom_input):
    """
    This function repeatedly asks the user for confirmation to either continue or restart the process.
    """
    while True:
        confirmation = input(f"Do you want to continue{custom_input}? y/N (or q for quit script): ").strip().lower()
        
        if confirmation == "y":
            print("Continuing...")
            return True  # Continue with the rest of the program
        elif confirmation == "q":
            print("Quitting the script...")
            #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
            print("\n\nExecution is being cancelled. Breaking the script.")
            sys.exit()  # Exit the script immediately
            #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
        else:
            print("Restarting the process...")
            return False  # Restart the process
            
#common visualistion section for QC
ask_to_check_visuals = "input" #"skip" --> skips the question

"evetuell presets lieber markiert in section"
#SECTION 1 CREATE THE BULK UNIT presets:
bulk_base = "Cu" # <--------------------------------------------------------------------- Save_file_name_relevant: bulk_base

#SECTION 2 INCLUDE THE ORIENTATION presets:


#SECTION 3 CUT THE SUPERCELL TO INCREASE READABILITY presets:


#SECTION 4 REPEAT THE BULK UNIT presets:
GB_full_revert = []

# Define the repetition factors
# repeats = [2, 4, 6, 8] # <--------------------------------------------------------------------- Save_file_name_relevant: repeat
repeats = [2]
#wideness = [1,3,5]
wideness = [1]

#SECTION 5 CREATE THE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE presets:


#SECTION 6 CREATE A SURFACE BY INCLUDING A VACUUM presets:


#SECTION 7 Delete 1 layer systematic approach -> Surfaces need to be equal on both sides presets:
#Step 0: set preset values: '#OPEN !!!!!!!!!


#presets:
#Step 1 (visualisation) preset:    #Options are: "input" "skip" 
#confirm_input1 = "input"
confirm_input1 = "skip"


#Step 2 (function) preset:         #Options are: "input" "skip" 
#Layer_delete_input = "input" 
Layer_delete_input = "skip" 
#use presets with "skip" instead of "input"
#selected side has to be L or R:
selected_side = "L"
#how many layers shoul be deleted:
#Nr_of_deleted_layers = 1
Nr_of_deleted_layers = 2

#Safety featutre that makes sure the layer is stoichometric (Not implemented yet)
#Step 3 (Safety feature) preset: WIP
define_stoichometric_layer = "skip" #Options are: "input" "skip"
stoichometric_layer_atom1 = 2 #Ga
stoichometric_layer_atom2 = 2 #Cu
stoichometric_layer_atom3 = 0 #Ag
stoichometric_layer_atom4 = 4 #Se

#step 4 (Safety feature) preset: WIP



#step 5 (function) preset:
""" already defined:
atoms 
selected_side 
Nr_of_deleted_layers 
"""
selection_tolerance = 0.5
visually_destinct_element = "Bi"


#step 6 (visualisation) preset:
#confirm_input6 = "input"
confirm_input6 = "skip"

#step 7 (function) preset:
#already done in step 5

#step 8 (visualisation) preset:
#confirm_input8 = "input"
confirm_input8 = "skip"
atom1 = "Ga"
atom2 = "Cu"
atom3 = "Ag"
atom4 = "Se"  

#step 9 (visualisation) preset:
confirm_input9 = "input"
#confirm_input9 = "skip"



#SECTION 8 INCLUDE AG OR VACANCYS presets:     
inclusion_atom_type = "Ag" # <--------------------------------------------------------------------- Save_file_name_relevant: inclusion_atom_type
inclusion_vacancy = True

#SECTION 9 INCLUDE xAG OR xCu presets:
#xincl = 50 # <--------------------------------------------------------------------- Save_file_name_relevant: xincl %
xincl = None



#--------------
#MAIN SECTION
#--------------




    
#SECTION 1 CREATE THE BULK UNIT

#bulk = ref_Cu.copy()
#bulk_base = "Cu"
#bulk_base = "Ag"
bulk = read(f"{project_dir}/input/ASE_Scripts_input/unit_cell/{bulk_base}GaSe2_rel.POSCAR")
#if repeat == 2:
    #view(bulk) #[CuGaSe2 Einheitszelle]
bulk.center()
#if repeat == 2:
    #view(bulk)

#SECTION 2 INCLUDE THE ORIENTATION

skip_section = ask_to_skip_section(Section = 'Section_2: "INCLUDE THE ORIENTATION"', skip_section = "no") #skip_section_q = "ask" "skip" "no" "edit"
if skip_section != "skip":
        
    def set_tilt_angle():
        """
        This function represents the main content that you want to run repeatedly based on confirmation.
        """
        print("Starting the process...")

        "Change to take user input:"
        # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped
        hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511
        gb_type = "S9" # <--------------------------------------------------------------------- Save_file_name_relevant: gb_type
        tilt_angle = None # <--------------------------------------------------------------------- Save_file_name_relevant: tilt_angle_in_deg
        #layers = input("Amount of Layers: ").strip().lower()
        layers = 12
        bulk = globals().get("bulk", None)
        GB = surface(bulk, hkl, layers, vacuum=0.0)
        a1, b1, c1 = GB.get_cell()
        GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
        GB.center()
        print(f"ask_user_for_visuals for Step: 2 INCLUDE THE ORIENTATION and SET THE LAYER AMOUNT")
        ask_user_for_visuals(GB, ask_to_check_visuals)
        # ------------------------------------------------------------------------------------------------------------|#looped
    
        while True:
            # Now we call the confirmation question to ask if the user wants to continue or restart
            continue_process = Confirmation_question(custom_input = "")

            if continue_process:
                print("Proceeding with the next step...")
                return GB
                break  # Exit loop if the user confirms to continue
            else:
                print("Restarting the process...")  # Just print and loop again if restarting

    if skip_section == "edit":
        # Running the edit process
        GB = set_tilt_angle()
        #print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
        #GB.edit()
        
    else:
        # Run the preset:
        #------------------------------------------------------------------------------------------------------|#preset
        hkl = (1, -1, 4) #[woher] --> https://www.sciencedirect.com/science/article/pii/S1359645416305511
        gb_type = "S9" # <--------------------------------------------------------------------- Save_file_name_relevant: gb_type
        tilt_angle = None # <--------------------------------------------------------------------- Save_file_name_relevant: tilt_angle_in_deg
        #layers = 36 #old ver 009
        layers = 12
        GB = surface(bulk, hkl, layers, vacuum=0.0)
        a1, b1, c1 = GB.get_cell()
        GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,1]])
        GB.center()
        #------------------------------------------------------------------------------------------------------|#preset
    

#SECTION 3 CUT THE SUPERCELL TO INCREASE READABILITY
skip_section = ask_to_skip_section(Section = 'Section_3: "CUT THE SUPERCELL TO INCREASE READABILITY"', skip_section = "no") #skip_section_q = "ask" "skip" "no" "edit"
if skip_section != "skip":
        
    def cut_supercell():
        """
        This function represents the main content that you want to run repeatedly based on confirmation.
        """
        print("Starting the process...")

        "Change to take user input:"
        # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped
        GB = globals().get("GB", None)
        GB_revert = copy.deepcopy(GB)
        #view(GB)
        default = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00))
        default_text = "cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00))"
        user_input = input(f"Enter cut operation e.g. [{default_text}]: ").strip()
        #Note: eval is simplest solution, but unsafe if user input is untrusted!
        #GB = eval(user_input) if user_input else default

        # Allowed variables and functions
        safe_globals = {
        "__builtins__": {},  # remove access to all built-in functions
        "cut": cut,
        "GB": GB
        }
        try:
            # Safely evaluate the user 
            if user_input:
                GB = eval(user_input, safe_globals)
            else:
                GB = default
        except Exception as e:
            print("❌ Error:", e)
        
        print("Your change:", user_input)
        GB.wrap()
        print(f"ask_user_for_visuals for Step: 3 CUT THE SUPERCELL TO INCREASE READABILITY")
        ask_user_for_visuals(GB, ask_to_check_visuals)
        flipped = "BULK" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped
        # ------------------------------------------------------------------------------------------------------------|#looped
    
        while True:
            # Now we call the confirmation question to ask if the user wants to continue or restart
            continue_process = Confirmation_question(custom_input = "")

            if continue_process:
                print("Proceeding with the next step...")
                return GB
                break  # Exit loop if the user confirms to continue
            else:
                print("Restarting the process...")  # Just print and loop again if restarting
                GB = copy.deepcopy(GB_revert)
                GB = cut_supercell()
    
    if skip_section == "edit":
        # Running the edit process
        print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
        GB = cut_supercell()
        GB.edit()
    else:
        # Run the preset:
        #------------------------------------------------------------------------------------------------------|#preset
        GB = cut(GB, a=(1, 0, 0), b=(-2.00,1.00, 0), c=(0, 0, 1.00)) ##old ver 009
        GB.wrap()
        flipped = "BULK" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped
        GB_bulk = copy.deepcopy(GB)
        #------------------------------------------------------------------------------------------------------|#preset

    print(f"save_functionality step 3: Base_supercell")
    #print(f"Includes: {bulk_base}GaSe2_BULKraw_{tilt_angle}")
    print(f"layers:{layers}")
    print(f"layer_nr:{layer_nr}")
    save_model(Section = "Section_3", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")


#SECTION 4.1 REPEAT THE BULK UNIT
skip_section = ask_to_skip_section(Section = "Section 4", skip_section = "no") #skip_section_q = "ask" "skip" "no"
if skip_section != "skip":
    
    #define the repetitions, the wideness (WIP)
    def define_supercell():
        """
        This function represents the main content that you want to run repeatedly based on confirmation.
        """
        print("Starting the process...")

        #WIP WIP WIP WIP WIP WIP WIP WIP
    
        "Change to take user input:"
        # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped
    
        
        # Define the repetition factors
        do = "nothing" #filler so that the cell does not remain empty
        # repeats = [2, 4, 6, 8]
        repeats = [2]
        # wideness = [1,3,5]
    
    
        
        # ------------------------------------------------------------------------------------------------------------|#looped
    
        while True:
            # Now we call the confirmation loop to ask if the user wants to continue or restart
            continue_process = Confirmation_loop()
    
            if continue_process:
                print("Proceeding with the next step...")
                break  # Exit loop if the user confirms to continue
            else:
                print("Restarting the process...")  # Just print and loop again if restarting
    
    if skip_section == "edit":
        # Running the edit process
        define_supercell()
    else:
        # Run the preset:
        #------------------------------------------------------------------------------------------------------|#preset
        # Define the repetition factors
        do = "nothing" #filler so that the cell does not remain empty
        #repeats = [2, 4, 6, 8]
        #repeats = [6]
        #repeats = [2, 4]
        repeats = [2]
        # wideness = [1,3,5]
        #wide = 3                                                                   #<--------------------------------------------------WIDE
        #------------------------------------------------------------------------------------------------------|#preset

#SECTION 4.2 REPEAT THE BULK UNIT
for repeat in repeats: # Loop over the repetition factors
    if GB_full_revert:
        GB = copy.deepcopy(GB_full_revert)
    GB_full_revert = copy.deepcopy(GB)
    layer_nr = int((repeat/2)*layers) # <--------------------------------------------------------------------- Save_file_name_relevant: layer_nr
    #print(layer_nr) 
    GB = GB_bulk.repeat((1, 1, repeat)) #[bulk size on both sides] 
    #for wide in wideness
    #GB = GB.repeat((wide, wide, repeat)) #[bulk size on both sides] ZU VARIIREN III! #<-----------------------------------------------WIDE
    GB.wrap()

    print(f"\nsave_functionality step 4: rep_supercell")
    #print(f"Includes: {bulk_base}GaSe2_BULK_{tilt_angle}_{repeat}rep")
    save_model(Section = "Section_4", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")

    #-------------------------------

    #SECTION 5 CREATE THE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE
    skip_section = ask_to_skip_section(Section = 'Section_5: "CREATE THE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE"', skip_section = "ask") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":

        def create_GB_middle():
            """
            This function represents the main content that you want to run repeatedly based on confirmation.
            """
            print("Starting the process...")

            #WIP WIP WIP WIP WIP WIP WIP WIP WIP still buggy

            "Change to take user input:"
            # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped

            GB = globals().get("GB", None)
            GB_revert = copy.deepcopy(GB)
        
            # Flip half of it [independent of bulk size]
            pos = GB.get_scaled_positions()
            center = 0.5

            default1 = center + (0.005 / (repeat / 2))  # tolerance threshold for the copy process (there is a danger to copy not enought/too many atoms)
            default2 = 1 * (center + (0.005 / (repeat / 2)) - 0.00 * pos[0, 2])  # position adjustment to close the gap
            
            # Show default expressions as strings for user guidance
            default1_text = "center + (0.005 / (repeat / 2))"
            default2_text = "1 * (center + (0.005 / (repeat / 2)) - 0.00 * pos[0, 2])"

            print("\nAdjust the tolerance threshold for the copy process. (there is a danger to copy not enought/too many atoms)")
            user_input1 = input(f"Enter input1 for z-copy_threshold [{default1_text}]: ").strip()
            print("Position adjustment to close the gap")
            user_input2 = input(f"Enter input2 for z-gap_adjustment [{default2_text}]: ").strip()
            
            # Safe namespace for eval (read-only access to needed vars)
            safe_globals = {
                "__builtins__": {},
                "center": center,
                "repeat": repeat,
                "pos": pos  # necessary if the user wants to use pos[0,2] etc.
            }
            
            try:
                input1 = eval(user_input1, safe_globals) if user_input1 else default1
                input2 = eval(user_input2, safe_globals) if user_input2 else default2
            except Exception as e:
                print("❌ Error in input evaluation:", e)
                input1 = default1
                input2 = default2

            if user_input1:
                print("Your change to input 1:", user_input1)
            if user_input2:
                print("Your change to input 2:", user_input2)
            
            # Apply flipping logic using user-defined (or default) values
            for i in range(len(pos)):
                if pos[i, 2] <= input1:
                    pos[i, 2] = -pos[i, 2] + input2
            # Apply changes to the GB
            GB.set_scaled_positions(pos)
            
            #set_name_for_save_file:
            flipped = "GB" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped

            print(f"ask_user_for_visuals for Step: 5.1 CREATE THE MIDDLE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE")
            ask_user_for_visuals(GB, ask_to_check_visuals)
        
            # ------------------------------------------------------------------------------------------------------------|#looped
    
            while True:
                # Now we call the confirmation question to ask if the user wants to continue or restart
                continue_process = Confirmation_question(custom_input = "")

                if continue_process:
                    print("Proceeding with the next step...")
                    return GB
                    break  # Exit loop if the user confirms to continue
                else:
                    print("Restarting the process...")  # Just print and loop again if restarting
                    GB = copy.deepcopy(GB_revert)
                    GB = create_GB_middle()
                    
        def create_GB_outside():
            """
            This function represents the main content that you want to run repeatedly based on confirmation.
            """
            print("Starting the process...")

            "Change to take user input:"
            # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped

            GB = globals().get("GB", None)
            GB_revert = copy.deepcopy(GB)
        
            # Allowed variables and functions
            safe_globals = {
            "__builtins__": {},  # remove access to all built-in functions
            "cut": cut,
            "GB": GB
            }
            try:
                # Safely evaluate the user 
                if user_input:
                    GB = eval(user_input, safe_globals)
                else:
                    GB = default
            except Exception as e:
                print("❌ Error:", e)
            
            print("Your change:", user_input)

            #5.2 These 2 cell parameter changes are to adjust the 2nd GB that results from repating the supercell with no vacuum
            #not neccesary to modify if a surface is created in section 6:

            #GB.get_cell() gets the simulation cell (a 3x3 matrix).
            #C1[2,2] is the length of the cell along the [z-axis] → it's increased by 2.0 units.
            #Then GB.center() shifts the structure to center it inside the new, larger cell.
            C1 = GB.get_cell()
            C1[2,2] = C1[2,2] + 2.0
            GB.set_cell(C1)
            GB.center()
        
            #This part reduces the z-length by 2 right after increasing it
            #wrap() ensures all atoms are inside the simulation box after all these changes.
            GB.set_cell(GB.get_cell() - [[0, 0, 0], [0,0,0], [0,0,2]])
            GB.center()
            GB.wrap()

            #set_name_for_save_file:
            flipped = "GB" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped

            print(f"ask_user_for_visuals for Step: 5.2 CREATE THE OUTSIDE GRAIN BOUNDARY BY ADJUSTING THE CELL PARAMETERS")
            ask_user_for_visuals(GB, ask_to_check_visuals)
        
            # ------------------------------------------------------------------------------------------------------------|#looped
    
            while True:
                # Now we call the confirmation question to ask if the user wants to continue or restart
                continue_process = Confirmation_question(custom_input = "")

                if continue_process:
                    print("Proceeding with the next step...")
                    return GB
                    break  # Exit loop if the user confirms to continue
                else:
                    print("Restarting the process...")  # Just print and loop again if restarting
                    GB = copy.deepcopy(GB_revert)
                    GB = create_GB_outside()
                    
        if skip_section == "edit":
            # Running the edit process
            print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
            GB = create_GB_middle()
            ask_to_check_outside_GB = input("Edit outside GB? (Not nessecary if Surface will be created later) y/N: ").strip().lower()
            if ask_to_check_outside_GB == 'y':
                GB = create_GB_outside()
            GB.edit()

        else:
            # Run the preset:
            #------------------------------------------------------------------------------------------------------|#preset
            
            # 5.1 Flip half of it [independent of bulk size]
            pos = GB.get_scaled_positions()
            center = 0.5
            #preset for 12 layers
            for i in range(len(pos)):
                if pos[i, 2] <= center+(0.010/(repeat/2)): #0,010 toleranzfaktor -> gefahr besteht ein Atom zu flippen/nicht zu flippen
                    pos[i, 2] = -pos[i, 2] + 1*(center + (0.018/(repeat/2)) - 0.00*pos[i, 2]) #justieren zum schließen der lücke
            GB.set_scaled_positions(pos)

            """ #preset for 36 layers
            for i in range(len(pos)):
                if pos[i, 2] <= center+(0.005/(repeat/2)): #0,005 toleranzfaktor -> gefahr besteht ein Atom zu flippen/nicht zu flippen
                    pos[i, 2] = -pos[i, 2] + 1*(center + (0.005/(repeat/2)) - 0.00*pos[i, 2]) #justieren zum schließen der lücke
            GB.set_scaled_positions(pos)
            """
            #if repeat == 4:
            #    view(GB)
            #view(GB)
            #time.sleep(1) # wait 1s

            #5.2 These 2 cell parameter changes are to adjust the 2nd GB that results from repating the supercell with no vacuum
            #not neccesary to modify if a surface is created in section 6:

            #GB.get_cell() gets the simulation cell (a 3x3 matrix).
            #C1[2,2] is the length of the cell along the [z-axis] → it's increased by 2.0 units.
            #Then GB.center() shifts the structure to center it inside the new, larger cell.
            C1 = GB.get_cell()
            C1[2,2] = C1[2,2] + 2.0
            GB.set_cell(C1)
            GB.center()

            #view(GB)
            #time.sleep(1) # wait 1s

            #This part reduces the z-length by 2 right after increasing it
            #wrap() ensures all atoms are inside the simulation box after all these changes.
            GB.set_cell(GB.get_cell() - [[0, 0, 0], [0,0,0], [0,0,2]])
            GB.center()
            GB.wrap()

            

            #view(GB)
            #time.sleep(1) # wait 1s

            #set_name_for_save_file:
            flipped = "GB" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped

            #------------------------------------------------------------------------------------------------------|#preset    
       

        #if repeat == 4:
        #   view(GB)
        #view(GB)
    else:
        print("This is the result of skipping section 5.")
        #GB.edit()
        ask_user_for_visuals(GB, ask_to_check_visuals)
        flipped = "BULK" # <--------------------------------------------------------------------- Save_file_name_relevant: flipped

    print(f"\nsave_functionality step 5: rep_GB")
    #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep")
    save_model(Section = "Section_5", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")
    
    #--------
    
    #SECTION 6 CREATE A SURFACE BY INCLUDING A VACUUM
    skip_section = ask_to_skip_section(Section = "Section 6", skip_section = "ask") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":

        def create_surface():
            """
            This function represents the main content that you want to run repeatedly based on confirmation.
            """
            print("Starting the process...")

            #WIP WIP WIP WIP WIP WIP WIP WIP WIP
            
            "Change to take user input:"
            # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped
        
            #Add a vaccum on both sides of the cell to create a surface
            #increase cell size by 40Å in z direction
            #print(GB.get_cell())
            #GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,20]])
            GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,40]]) #mehr Vaccum test
            #print(GB.get_cell())
            #if repeat == 2:
            #    view(GB)
            #center the atoms to create a 20Å Vaccum on both sides of the model
            GB.center()
            
            # ------------------------------------------------------------------------------------------------------------|#looped
    
            while True:
                # Now we call the confirmation question to ask if the user wants to continue or restart
                continue_process = Confirmation_question(custom_input = "")

                if continue_process:
                    print("Proceeding with the next step...")
                    break  # Exit loop if the user confirms to continue
                else:
                    print("Restarting the process...")  # Just print and loop again if restarting
    
        if skip_section == "edit":
            # Running the edit process
            create_surface()
        else:
            # Run the preset:
            #------------------------------------------------------------------------------------------------------|#preset
            
            #Add a vaccum on both sides of the cell to create a surface
            #increase cell size by 40Å in z direction
            #print(GB.get_cell())
            #GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,20]])
            vacuums = 20
            GB.set_cell(GB.get_cell() + [[0, 0, 0], [0,0,0], [0,0,vacuums*2]]) #mehr Vaccum test
            #print(GB.get_cell())
            #if repeat == 2:
            #    view(GB)
            #center the atoms to create a 20Å Vaccum on both sides of the model
            GB.center()

            #------------------------------------------------------------------------------------------------------|#preset    
        
        print(f"\nsave_functionality step 6: vacuums_supercell")
        #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_TODOVACUUMSraw")
        save_model(Section = "Section_6", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")

    else:
        do = "nothing"
        #print("This is the result of skipping section 6.")
        #GB.edit()
    
    #---------------------------------------------------------------------
    
    #SECTION 7 Delete 1 layer systematic approach -> Surfaces need to be equal on both sides 
    skip_section = ask_to_skip_section(Section = "Section 7", skip_section = "ask") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":
        
        #Step 1 ask: show model | visualisation
        if confirm_input1 == "skip":
            do = "nothing"
            #print("skipping Step 1. Script continues.")
        else:
            confirm_input1 = input("show model? y/N: ").strip().lower()
            
            if confirm_input1 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
            else:
                print("Step 1 Done. Script continues.")
    
    
        # Step 2 ask to delete layers: by calling function: get_layer_delete_input() | function
        if Layer_delete_input == "input": 
            selected_side, Nr_of_deleted_layers = get_layer_delete_input()
            print("Step 2 Done. Script continues.")
            #Layer_delete_input = input("Delete a stoichometric layer? (No[default]/left/right/custom): N/l/r/c")#<--------------------------
            #custom Format: (Nr.)_L/R
        else:
            #keep preset
            do = "nothing"
            #print("Step 2 Done. Script continues.")
    
    
        #Step 3 define a stoichometric layer for comparision --> static / user input #OPEN !!!!!!!!! | Safety feature
        
        if define_stoichometric_layer == "skip":
            stoichometric_layer_list = [stoichometric_layer_atom1,stoichometric_layer_atom2,stoichometric_layer_atom3,stoichometric_layer_atom4]
        else:
            stoichometric_layer_atom1 = input(f"Atom count {atom1}:")
            stoichometric_layer_atom2 = input(f"Atom count {atom2}:")
            stoichometric_layer_atom3 = input(f"Atom count {atom3}:")
            stoichometric_layer_atom4 = input(f"Atom count {atom4}:")
            stoichometric_layer_list = [stoichometric_layer_atom1,stoichometric_layer_atom2,stoichometric_layer_atom3,stoichometric_layer_atom4]
            print(stoichometric_layer_list)
            
            print("Step 3 Done. Script continues.")
    
    
        #step 4 identify the atom index in the layer and compare to stoichometric layer #OPEN !!!!!!!!! | Safety feature
        
        #find smallest/L lagrgest/R z-Axis position
    
        #write indizes in list
        deletion_indices_list = []
        #define:
        deletion_layer_atom1 = 2 #Ga
        deletion_layer_atom2 = 2 #Cu
        deletion_layer_atom3 = 0 #Ag
        deletion_layer_atom4 = 4 #Se
        #if lists are equal: interchannge atoms with [visually_destinct_Element = Bi]
        #print("Step 4 Done. Script continues.")
    
        
        #step 5 interchannge atoms with [visually_destinct_Element = Bi] --> Large und Purple Call the function find_and_swap_atoms | function
        
        GB = find_and_swap_atoms(GB, selected_side, Nr_of_deleted_layers, selection_tolerance, visually_destinct_element)
        if Layer_delete_input != "skip":
            print("Step 5 Done. Script continues.")
    
    
        #step 6 save/show model and ask for confirmation Y/n | visualisation
    
        #ask for confirmation: # Default behavior is skip
        if confirm_input6 == "skip":
            do = "nothing"
            #print("skipping Step 6. Script continues.")
        else:
            confirm_input6 = input("Check if the correct atoms selected? y/N: ").strip().lower()
            
            if confirm_input6 == 'y':
                #step 5.5 if y --> If possible manually fix it in ASE window 
                print("Script will continue after closing the ASE gui window.\nEdits will be saved\nplease correct the deletion layer using the Ctrl+Y feature in ASE")
                GB.edit()
            else:
                print("Skipping correction.")
        
        #Save the result with a distinct filename #needs to be corrected
        path_to_inbetweens = f"{project_dir}/data/ASE_Inbetween_steps"
        filename = f"{path_to_inbetweens}/Custom_model_skript/GB-S9-CuGaSe2-{repeat}rep_10A_VACUUMS_PLACEHOLDER_TEMP_NAME.vasp"
        GB.write(filename, vasp5=True, sort=True, direct=True)
        print(f"Saved: {filename}")
        
        if Layer_delete_input != "skip":
            print("Step 6 Done. Script continues.")
    
    
        #step 7 delete visually_destinct_Element from modell | function
        
        GB = delete_atoms_by_element(GB, visually_destinct_element)
        if Layer_delete_input != "skip":
            print("Step 7 Done. Script continues.")
    
    
        #step 8 give Total atom count and ask to show final version y/N | visualisation
    
        #Give out Atom count and Formation Energy (maybe rewrite)
        if Layer_delete_input != "skip":
            Number_Atoms = list(GB.symbols)
            Number_Atoms_count1 = Number_Atoms.count(atom1)
            Number_Atoms_count2 = Number_Atoms.count(atom2)
            Number_Atoms_count3 = Number_Atoms.count(atom3)
            Number_Atoms_count4 = Number_Atoms.count(atom4)
            print(f"Atom count {atom1}: {Number_Atoms_count1}") #Ga
            print(f"Atom count {atom2}: {Number_Atoms_count2}") #Cu
            print(f"Atom count {atom3}: {Number_Atoms_count3}") #Ag
            print(f"Atom count {atom4}: {Number_Atoms_count4}") #Se
    
        if confirm_input8 == "skip":
            do = "nothing"
            #print("skipping Step 8. Script continues.")
        else:
            confirm_input8 = input("Show deletion result? y/N: ").strip().lower()
            
            if confirm_input8 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
                print("Step 8 Done. Script continues.")
            else:
                print("Skipping correction.")
        
        
        #step 9 ask delete new Layer? y/N
    
        if confirm_input9 == "skip":
            do = "nothing"
            #print("skipping Step 9. Script continues.")
        else:
            confirm_input9 = input("delete more layers? y/N: ").strip().lower()
            
            if confirm_input9 == 'y':
                print("Script will continue after closing the ASE gui window.\nEdits will be saved.")
                GB.edit()
            else:
                print("")
                
        if Layer_delete_input != "skip":
            if Nr_of_deleted_layers == 0:
                print(f"No layer deleted.")
            elif Nr_of_deleted_layers == 1:
                print(f"Deletion of {Nr_of_deleted_layers} layer complete.")
            else:
                print(f"Deletion of {Nr_of_deleted_layers} layers complete.")

        print(f"\nsave_functionality step 7: improve_surface_supercell")
        #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_TODOVACUUMS(surface_edit/clean)")
        save_model(Section = "Section_7", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")
        
    else:
        do = "nothing"
        #print("This is the result of skipping section 7.")
        #GB.edit()


    
    #---------------------------------------------------------------------
    
    #SECTION 8 INCLUDE AG OR VACANCYS
    skip_section = ask_to_skip_section(Section = "Section 8", skip_section = "ask") #skip_section_q = "ask" "skip" "no"
    if skip_section != "skip":
    
        "define functions: (needs to be moved after testing):"
        
    
    
        
        # Step 8.0 index Cu center to border:
        """
        print("edit 1")
        print(GB)
        """
        # Problem: Shallow Copy: GB_revert = GB creates a shallow copy of GB. 
        # This means both GB_revert and GB reference the same object (atoms list).
        # Solution:
        # Create a deep copy of GB
        GB_revert = copy.deepcopy(GB)  # Make sure GB_revert is an independent copy
        """
        print(GB_revert)
        GB.edit()
        """
    
        # Step 8.1 find center
        center = find_center_of_atoms(GB)
        
        # Step 8.2 find Cu pos closest to center
        closest_atom_index, closest_atom_position = find_closest_atom_to_center(GB, center, "Cu")
        #print(f"TEST: Closest Cu atom is at index {closest_atom_index} with coordinates: {closest_atom_position}")
        
        # Step 8.2.1 confirm Cu (replace with Bi)
        GB_test1 = swap_atoms(GB, closest_atom_index, "Bi")
        print(f"ask_user_for_visuals for Step: 8.2.1 confirm Cu (replace with Bi)")
        ask_user_for_visuals(GB_test1, ask_to_check_visuals)
        
        # Reset the symbols to the original (using the deep copy)
        GB = copy.deepcopy(GB_revert)  # Revert back to the original state
    
        """
        print("edit 2")
        print(GB_revert)
        print(GB)
        GB.edit()
    
        print("\n\nExecution is being cancelled 1. Breaking the script.")
        sys.exit()  # Exit the script immediately
        """
        
    
    
        
        #Step 8.3 find Cu pos > center (in x or y tolerance room? 1 [x] strict \ \ \ \ pattern should not matter --> periodicity)
        #and create index list: distance_to_center_index_list 
        """
        #distance_to_center_index_list = [closest_atom_index]
        #distance_to_center_index_list.append(next_atom_index)
        #distance_to_center_index_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        """
        
        #Step 1.4 create index list: distance_to_center_index_list 
        distance_to_center_index_list = []
        distance_to_center_index_list_1 = []
        distance_to_center_index_list_2 = []


        
        #def get_atoms_with_smaller_z(atoms, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance=0.1):
        distance_to_center_index_list_1 = get_atoms_with_smaller_z(GB, closest_atom_position, closest_atom_index, bulk_base="Cu", y_tolerance= 0.16, x_tolerance = 3) 
        distance_to_center_index_list_1.reverse()
        print(f"distance_to_center_index_list_part1 = {distance_to_center_index_list_1}")

        closest_atom_index_list = [closest_atom_index]
        print(f"closest_atom_index_list (to center) = {closest_atom_index_list}")
    
        distance_to_center_index_list_2 = get_atoms_with_larger_z(GB, closest_atom_position, closest_atom_index, bulk_base = "Cu", y_tolerance = 0.16, x_tolerance = 3)    
        print(f"distance_to_center_index_list_part2 = {distance_to_center_index_list_2}")
    
        distance_to_center_index_list = distance_to_center_index_list_1 + closest_atom_index_list + distance_to_center_index_list_2
        print(f"distance_to_center_index_list = {distance_to_center_index_list}")

        #uncomment for one incl in center
        #distance_to_center_index_list = closest_atom_index_list
        
    
        #Step 8.4 test version --> untill surface is hit (save atoms object as GB_incl_long)
        GB_test2 = swap_atoms(GB, distance_to_center_index_list, "Bi")
        #view(GB_test2)
        print(f"ask_user_for_visuals for Step: 8.4 test version --> untill surface is hit")
        ask_user_for_visuals(GB_test2, ask_to_check_visuals)
        # Reset the symbols to the original (using the deep copy)
        GB = copy.deepcopy(GB_revert)  # Revert back to the original state
        #print("edit 3")
        #GB.edit()

        """
        #Step 8.5.1 cut index list down to half of the remaining cell --> at once
        #distance_to_center_index_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # Calculate the halfway point
        halfway_index = len(distance_to_center_index_list) // 2
        # Slice the list to get the first half
        first_half = distance_to_center_index_list[:halfway_index]
        distance_to_center_index_list = first_half
        print("First half of the list:", distance_to_center_index_list)
        GB_test3 = swap_atoms(GB, distance_to_center_index_list, "Bi")
        #view(GB_test3)
        print(f"ask_user_for_visuals for Step: 8.5.1 cut index list down to half of the remaining cell")
        ask_user_for_visuals(GB_test3, ask_to_check_visuals)
        # Reset the symbols to the original (using the deep copy)
        GB = copy.deepcopy(GB_revert)  # Revert back to the original state
        #print("edit 4")
        #GB.edit()
        """
    
        """
        #Step 8.5.2 cut index list down to half of the remaining cell + extra 2
        # Calculate the halfway point
        halfway_index = len(distance_to_center_index_list) // 2
        print(f"halfway_index:{halfway_index}")
        halfway_index = halfway_index + 2
        print(f"halfway_index:{halfway_index}")
        # Slice the list to get the first half
        print(f"distance_to_center_index_list:{distance_to_center_index_list} ,length: {len(distance_to_center_index_list)}")
        first_half = distance_to_center_index_list[:halfway_index]
        distance_to_center_index_list = first_half
        print("First half of the list:", distance_to_center_index_list)
        print(f"distance_to_center_index_list:{distance_to_center_index_list} ,length: {len(distance_to_center_index_list)}")
        GB_test3 = swap_atoms(GB, distance_to_center_index_list, "Bi")
        #view(GB_test3)
        print(f"ask_user_for_visuals for Step: 8.5.1 cut index list down to half of the remaining cell")
        ask_user_for_visuals(GB_test3, ask_to_check_visuals)
        # Reset the symbols to the original (using the deep copy)
        GB = copy.deepcopy(GB_revert)  # Revert back to the original state
        #print("edit 4")
        #GB.edit()
        """  

        """
        #Step 8.5.3 cut index list down to half of the remaining cell --> periodicity every 2nd
        print(distance_to_center_index_list)
        half_list = [x for x in distance_to_center_index_list if x % 2 != 0]   
        print(half_list)
        GB_test4 = swap_atoms(GB, half_list, "Bi")
        #view(GB_test4)
        print(f"ask_user_for_visuals for Step: 1.6.2")
        ask_user_for_visuals(GB_test3, ask_to_check_visuals)
        # Reset the symbols to the original (using the deep copy)
        GB = copy.deepcopy(GB_revert)  # Revert back to the original state
        #print("edit 5")
        #GB.edit()
        """  
    
        #Step 8.6 systematicly SWAP or DEL
        inclusion_atom_type = "Ag" # <--------------------------------------------------------------------- Save_file_name_relevant: inclusion_atom_type
        inclusion_atom_type_reset = copy.deepcopy(inclusion_atom_type)
        inclusion_vacancy = True 
        position_index = 1
        
        for index in distance_to_center_index_list:
            #Step 8.6.1
            #systematicly SWAP replace 1 + save vasp [Ag]
            GB = swap_atoms(GB, index, inclusion_atom_type)
            """
            if index == distance_to_center_index_list[2]:
                print(f"ask_user_for_visuals for Step: 8.6.1 systematicly SWAP replace 1 + save vasp [Ag]")
                ask_user_for_visuals(GB, ask_to_check_visuals)
            """
            print(f"\nsave_functionality step 8.1: inclusion_supercell")
            #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_TODOVACUUMS(if)_Pos{position_index}-{inclusion_atom_type}")
            #save_functionality("Section 8.1", save_in = "input")
            save_model(Section = "Section_8-1", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")
            
            #Step 8.6.2
            if inclusion_vacancy == True:
                #systematicly DEL delete 1 + save vasp [vac]
                GB = delete_atoms_by_element(GB, inclusion_atom_type)
                """
                if index == distance_to_center_index_list[2]:
                    print(f"ask_user_for_visuals for Step: 8.6.2 systematicly DEL 1 + save vasp [Vac]")
                    ask_user_for_visuals(GB, ask_to_check_visuals)
                """
            print(f"save_functionality step 8.2: vac_supercell")
            inclusion_atom_type = "Vac"
            #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_TODOVACUUMS(if)_Pos{position_index}-{inclusion_atom_type}")
            #save_functionality("Section 8.1", save_in = "input")
            save_model(Section = "Section_8-2", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")
            inclusion_atom_type = copy.deepcopy(inclusion_atom_type_reset)
    
            position_index += 1 # <--------------------------------------------------------------------- Save_file_name_relevant: position_index
            GB = copy.deepcopy(GB_revert)  # Revert back to the original state
    
        #include: GB.delete_atoms_by_element(GB , incl = "Ag")
    
        """
        reminder:
        def delete_atoms_by_element(atoms, visually_destinct_element):
        
        #Function to delete all atoms of a specific element from the Atoms object.
    
        #Parameters:
        #atoms (ASE Atoms object): The atoms object from which atoms will be deleted.
        #visually_destinct_element (str): The element symbol (e.g., 'Bi') that needs to be removed from atoms.
    
        #Returns:
        #Atoms: Updated Atoms object with the specified element deleted.
        
        # Create a list of indices to delete
        indices_to_delete = [i for i, atom in enumerate(atoms) if atom.symbol == visually_destinct_element]
        
        # Delete the atoms by the indices
        for index in sorted(indices_to_delete, reverse=True):  # Reverse sorting to avoid index shifting
            del atoms[index]
    
        return atoms
        """
    
        #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
        #print("\n\nExecution is being cancelled 3. Breaking the script.")
        #sys.exit()  # Exit the script immediately
        #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
    else:
        do = "nothing"    
        #print("This is the result of skipping section 8.")
        #GB.edit()

    #---------------------------------------------------------------------

    #SECTION 9 INCLUDE xAG OR xCu
    #QUESTION: DOES IT NEED TO BE SYMETRICAL??? --> if YES move section 
    skip_section = ask_to_skip_section(Section = "Section 9", skip_section = "ask") #skip_section_q = "ask" "skip" "no" "edit"
    if skip_section != "skip":

        # Create a deep copy of GB
        GB_revert = copy.deepcopy(GB)  # Make sure GB_revert is an independent copy
        
        def include_percentage():
            """
            This function represents the main content that you want to run repeatedly based on confirmation.
            """
            print("Starting the process...")

            "Change to take user input:"
            # Add any process code here that you want to repeat or continue ----------------------------------------------|#looped
    



        
            # ------------------------------------------------------------------------------------------------------------|#looped

            while True:
                # Now we call the confirmation loop to ask if the user wants to continue or restart
                continue_process = Confirmation_loop()

                if continue_process:
                    print("Proceeding with the next step...")
                    break  # Exit loop if the user confirms to continue
                else:
                    print("Restarting the process...")  # Just print and loop again if restarting

        if skip_section == "edit":
            # Running the edit process
            include_percentage()
        else:
            # Run the preset:
            #------------------------------------------------------------------------------------------------------|#preset

            #WIP WIP WIP WIP WIP WIP WIP WIP WIP

            #xincl_list = 50 # <--------------------------------------------------------------------- Save_file_name_relevant: xincl %
            xincl_list = [10,20,30,40,50,60,70,80,90,100]

           
            #Step 1 Select all Cu --> index in list
            replace_index_list = select_atoms_of_type(GB, bulk_base)
            print(f"{bulk_base}_Index list:{replace_index_list}")
            print(f"{bulk_base}_Index list lenght:{len(replace_index_list)}")

            #Step 2 Select x% with randomizer (seeded)
            import random

            """
            def remove_percentage_from_list(data_list, percentage, seed=None):
                if seed is not None:
                    random.seed(seed)

                num_to_remove = int(len(data_list) * (percentage / 100))
                #""
                it uses int(), which always rounds down (a.k.a. floor behavior). So:
                If your list has 10 items
                And you pass in 32.4%
                You get int(10 * 0.324) = int(3.24) = 3 items removed.
                If you want it to round to the nearest instead, you can use round()
                num_to_remove = round(len(data_list) * (percentage / 100))
                That way:
                32.4% of 10 → 3 items
                32.6% of 10 → 4 items
                #""
                indices_to_remove = random.sample(range(len(data_list)), num_to_remove)
    
                return [item for i, item in enumerate(data_list) if i not in indices_to_remove]

            # Example usage
            my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
            #result = remove_percentage_from_list(my_list, 30, seed=42)
            result = remove_percentage_from_list(replace_index_list, 30, seed=42)
            print(result)
            """
            
            #new pls check
            def remove_neg_percentage_from_list(data_list, percentage, seed=None):
                if not (0 <= percentage <= 100):
                    raise ValueError("Percentage must be between 0 and 100.")

                # If no seed is provided, generate a random one
                if seed is None:
                    seed = random.randint(0, 2**32 - 1)

                random.seed(seed)

                num_to_remove = round(len(data_list) * ((100-percentage) / 100))
                #print(f"num_to_remove:{num_to_remove}")
    
                if num_to_remove == 0:
                    return data_list.copy(), seed

                indices_to_remove = random.sample(range(len(data_list)), num_to_remove)
                #print(f"indices_to_remove:{indices_to_remove}")
    
    
                reduced_list = [item for i, item in enumerate(data_list) if i not in indices_to_remove]
                return reduced_list, seed

            # If percentages is a single value, make sure it’s treated as a list
            if isinstance(xincl_list, (int, float)):  # Check if it's a single integer or float
                xincl_list = [xincl_list]  # Convert it into a list

            # Loop over percentages from 10 to 100
            #for p in range(10, 101, 10):
            #for p in percentages:
            for xincl in xincl_list:
                result, seed = remove_neg_percentage_from_list(replace_index_list, xincl, seed=2025)  # fixed seed for reproducibility
                print(f"{xincl}% {bulk_base} removed -> Result: {result}, Seed: {seed}")
                reduced_index_list = result            
                
    
                #Step 4 SWAP
                GB = swap_atoms(GB, reduced_index_list, inclusion_atom_type)
                ask_user_for_visuals(GB, ask_to_check_visuals)
    
                #Extra step: make sure that the distribution is equal enough
                #(check if this is even neccessary)
    
                #Step 5 save as
                print(f"\nsave_functionality step 9: xpercent_supercell")
                #print(f"Includes: {bulk_base}GaSe2_{flipped}_{tilt_angle}_{repeat}rep_TODOVACUUMS(if)_{xincl}{inclusion_atom_type}.data")
                save_model(Section = "Section_9", print_save_path=True, custom_filename=False, extend_filename="", save_in="default")

                print(f"{xincl}{inclusion_atom_type} DONE\n")

                #Step 6 restore GB
                GB = copy.deepcopy(GB_revert)  # Revert back to the original state




            

            #------------------------------------------------------------------------------------------------------|#preset 
    


    #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
    #print("\n\nExecution is being cancelled 4. Breaking the script.")
    #sys.exit()  # Exit the script immediately
    #---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT---EXIT
    
    else:
        do = "nothing"
        #print("This is the result of skipping section 9.")
        #GB.edit()

    #-------------------------------------------------------------------------

    #Give out Atom count and Formation Energy (maybe rewrite)
    print("all sections done")
    ask_user_for_visuals(GB, ask_to_check_visuals)
    Number_Atoms = list(GB.symbols)
    Number_Atoms_count1 = Number_Atoms.count(atom1)
    Number_Atoms_count2 = Number_Atoms.count(atom2)
    Number_Atoms_count3 = Number_Atoms.count(atom3)
    Number_Atoms_count4 = Number_Atoms.count(atom4)
    print(f"Atom count {atom1}: {Number_Atoms_count1}") #Ga
    print(f"Atom count {atom2}: {Number_Atoms_count2}") #Cu
    print(f"Atom count {atom3}: {Number_Atoms_count3}") #Ag
    print(f"Atom count {atom4}: {Number_Atoms_count4}") #Se
    print(f"layer count: {layer_nr}layers") 
    #GB.set_calculator(pot)
    #print(gb_energy(GB, eref_Cu, eref_Ag, xAg=0)*16.0218)

    #reset a few name tags:
    flipped = "BULK"
    vacuums = None
    position_index = None
    inclusion_atom_type = None
    xincl = None
    
    print(f"{repeat}rep done\n\nNext:")



print("\nDONE")


# Get the current date and time
import datetime # Import the 'datetime' module to work with date and time
now = datetime.datetime.now()# Create a datetime object representing the current date and time
print("\nTimestamp:")# Display a message indicating what is being printed
print(now.strftime("%d-%m-%Y %H:%M"))# Print the current date and time in a specific format "%Y-%m-%d %H:%M:%S"




save_functionality step 3: Base_supercell
layers:12
layer_nr:None
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_3/CuGaSe2_BULK_S9_12z-layers.vasp

save_functionality step 4: rep_supercell
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_4/CuGaSe2_BULK_S9_12z-layers.vasp


Skip Section_5: "CREATE THE GRAIN BOUNDARY BY MIRRORING THE BULK IN THE MIDDLE" of the model creation prosses? y/N:  



save_functionality step 5: rep_GB
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_5/CuGaSe2_GB_S9_12z-layers.vasp


Skip Section 6 of the model creation prosses? y/N:  



save_functionality step 6: vacuums_supercell
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_6/CuGaSe2_GB_S9_12z-layers_20A-VACUUMS.vasp


...

Script will continue after closing the ASE gui window.
Edits will be saved
please correct the deletion layer using the Ctrl+Y feature in ASE
The variable is an integer: 119

save_functionality step 8.1: inclusion_supercell
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_8-1/CuGaSe2_GB_S9_12z-layers_20A-VACUUMS_Pos1-Ag.vasp
save_functionality step 8.2: vac_supercell
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_8-2/CuGaSe2_GB_S9_12z-layers_20A-VACUUMS_Pos1-Vac.vasp
The variable is an integer: 110

save_functionality step 8.1: inclusion_supercell
Saved: /nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/data/ASE_Inbetween_steps/Custom_model_skript/Section_8-1/CuGaSe2_GB_S9_12z-layers_20A-VACUUMS_Pos2-Ag.vasp
save_functionality step 8.2: vac_supercell
Saved: /nfshome/okresa/Bachelor_

Skip Section 9 of the model creation prosses? y/N:  y


Skipping Section 9 of the model creation prosses
all sections done


Check with visualisation? y/N:  


Skipping visualistion.
Atom count Ga: 90
Atom count Cu: 90
Atom count Ag: 0
Atom count Se: 180
layer count: 12layers
2rep done

Next:

DONE

Timestamp:
20-05-2025 18:25


In [14]:
"""
short script that reads out all .vasp files in the folder
/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures/
and changes silver atoms (Ag) to copper atoms (Cu)
"""

from ase.io import read, write
from pathlib import Path

# Define the target folder
input_folder = Path("/nfshome/okresa/Bachelor_Thesis_Office/Project_Vacancy_Intersetiell_ASE_model/input/LAMMPS_input/Active_input/POSCAR_Structures")

# Loop over all .vasp files
for poscar_file in input_folder.glob("*.vasp"):
    print(f"Processing: {poscar_file.name}")
    
    # Read structure
    atoms = read(poscar_file, format='vasp')
    
    # Replace Ag with Cu
    for atom in atoms:
        if atom.symbol == "Ag":
            atom.symbol = "Cu"
    
    # Create output path
    output_file = poscar_file.with_name(poscar_file.stem + "_modified.vasp")
    
    # Write the modified structure
    write(output_file, atoms, format='vasp')
    print(f"Written modified file: {output_file.name}")

Processing: CuGaSe2_GB_S9_24z-layers_3wide_20A-VACUUMS_Pos1-Ag.vasp
Written modified file: CuGaSe2_GB_S9_24z-layers_3wide_20A-VACUUMS_Pos1-Ag_modified.vasp
